# Read me

## please run the Exploratory and Association Rules Analysis first. This code requires pickles from the Association Rule Mining.

METADATA

trx_date: the date of the transaction;

trx_id: the ID of each transaction;

product_code: the ID of the product;

sub_family: subfamily is a product hierarchy;

family: family is a product hierarchy;

quantity: number of product units;

unit_price: the unit price of the product;

flag_pastry: boolean flag to identify products that belong to the pastry category;

shopID: the ID of the location/shop where the transaction occured;

customerID: the ID of the customer in the purchase transaction;

In [ ]:
!pip install openpyxl
!pip install mlxtend
!pip install networkx matplotlib
!pip install joypy
!pip install factor_analyzer


In [ ]:
def remove_outliers_zscore(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    filtered_data = data[abs(z_scores) <= threshold]
    return filtered_data


## defines a function that plots multiple box plots
def plot_multiple_boxplots(data, feats, title="Box Plots"):

    # Prepare figure. Create individual axes where each histogram will be placed
    fig, axes = plt.subplots(2, ceil(len(feats) / 2), figsize=(20, 11))

    # Plot data
    # Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
    for ax, feat in zip(axes.flatten(), feats): # Notice the zip() function and flatten() method
      sns.boxplot(x=data[feat], ax=ax)
    
    # Layout
    # Add a centered title to the figure:
    plt.suptitle(title)

    plt.show()

    return


def is_redundant(rule, rules_df):
    """
    Check if a given rule is redundant within a set of rules.
    A rule is redundant if there's another rule with the same consequent,
    equal or higher confidence, and a subset of the antecedents.
    """
    for _, other_rule in rules_df.iterrows():
        if (rule['consequents'] == other_rule['consequents']) and \
           (rule['antecedents'].issubset(other_rule['antecedents'])) and \
           (rule['confidence'] <= other_rule['confidence']) and \
           (rule['antecedents'] != other_rule['antecedents']):  # Avoid comparing a rule to itself
            return True
    return False




In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import ticker
import joypy
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from factor_analyzer.factor_analyzer import calculate_kmo
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import r2_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from math import ceil
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import matplotlib.colors as mcolors
from itertools import product
import pandas as pd
from matplotlib.colors import ListedColormap
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import re
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage as sch_linkage
import pickle
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
sales_data = pd.read_csv('anon_sales.csv')
sales_data.describe().T

# Part I: Data Pre-Cleaning

In [ ]:
# Load the dictionary from the file
with open('meta_dfs.pkl', 'rb') as file:
    meta_dfs = pickle.load(file)

In [ ]:
# Load the dictionary from the file
with open('meta_dfs_2023.pkl', 'rb') as file:
    meta_dfs_2023 = pickle.load(file)

In [ ]:
# Load the dictionary from the file
with open('meta_dfs_lift.pkl', 'rb') as file:
    meta_dfs_lift = pickle.load(file)

In [ ]:
def order_id_frozensets(df, columns=['antecedents', 'consequents']):
    """
    Orders the ID frozensets within specified columns of a DataFrame.

    Args:
    df (DataFrame): The DataFrame to process.
    columns (list of str): List of column names to order frozensets in.
    """
    for col in columns:
        # Convert frozenset to sorted list, then back to frozenset
        df[col] = df[col].apply(lambda x: frozenset(sorted(x)))
    return df

# Apply the function to each DataFrame in meta_dfs
for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        meta_dfs[shop][year] = order_id_frozensets(df, columns=['antecedents', 'consequents'])

# Now, meta_dfs has been updated with ordered frozensets in the specified columns.


In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['family'] = sales_data['family'].str.replace('family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['family'] = pd.to_numeric(sales_data['family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['sub_family'] = sales_data['sub_family'].str.replace('sub_family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['sub_family'] = pd.to_numeric(sales_data['sub_family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['shopID'] = sales_data['shopID'].str.replace('ShopID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['shopID'] = pd.to_numeric(sales_data['shopID'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['product_code'] = sales_data['product_code'].str.replace('Product_', '')

In [ ]:
# Replace 'trx_id' with '' (empty string) to leave only the number part
sales_data['trx_id'] = sales_data['trx_id'].str.replace('trxID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['trx_id'] = pd.to_numeric(sales_data['trx_id'])

In [ ]:
# Convert the 'trx_date' column to datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

In [ ]:
sales_data['unit_price'] = pd.to_numeric(sales_data['unit_price'].str.replace(',', '.'), errors='coerce')

In [ ]:
# Create the new variable total_value as the product of quantity and unit_price
sales_data['total_value'] = sales_data['quantity'] * sales_data['unit_price']

In [ ]:
# Mapping from original years to new years
year_mapping = {
    1992: 2020,
    1993: 2021,
    1994: 2022,
    1995: 2023,
    1996: 2024
}

# Apply the mapping to change the year, keeping month and day the same
sales_data['trx_date'] = sales_data['trx_date'].apply(
    lambda x: x.replace(year=year_mapping[x.year])
)

sales_data['trx_date']

In [ ]:
# Create a 'year' column based on 'trx_date'
sales_data['year'] = sales_data['trx_date'].dt.to_period('Y')

# Create a 'quarter' column based on 'trx_date'
sales_data['quarter'] = sales_data['trx_date'].dt.to_period('Q')

In [ ]:
# Create 'year_integer' column by extracting year and converting to integer
sales_data['year_integer'] = sales_data['trx_date'].dt.year


In [ ]:
# Determine season
def get_season(month):
    seasons = {'winter': (12, 1, 2), 'spring': (3, 4, 5), 'summer': (6, 7, 8), 'autumn': (9, 10, 11)}
    for season, months in seasons.items():
        if month in months:
            return season
    return None

sales_data['season'] = sales_data['trx_date'].dt.month.apply(get_season)

In [ ]:
# Day of the week (Monday=1, Tuesday=2, ...)
sales_data['day_of_week_number'] = sales_data['trx_date'].dt.dayofweek + 1

In [ ]:
# Day of the week name
sales_data['day_of_week_name'] = sales_data['trx_date'].dt.day_name()

In [ ]:
sales_data['year_month'] = sales_data['trx_date'].dt.to_period('M')

In [ ]:
# Assuming meta_dfs is your dictionary containing all the shop dataframes
all_data = []
for shop, years in meta_dfs.items():
    for year, df in years.items():
        df['shop'] = shop
        df['year'] = year
        all_data.append(df)

# Concatenate all the dataframes into one
combined_data = pd.concat(all_data, ignore_index=True)

# Create 'rule' column if it doesn't exist
combined_data['rule'] = combined_data['LHS'] + ' → ' + combined_data['RHS']

In [ ]:
combined_data.head()

In [ ]:
# Initialize a dictionary to store the support values for each rule
rule_supports = {}

# Iterate over each shop and year to extract support values for each rule
for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        # Iterate through each row to extract rule support
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            # Initialize or update the support for the rule in the specific shop
            rule_supports.setdefault(rule_identifier, {})[shop] = row['support']

# Convert the rule supports into a DataFrame for easier manipulation
supports_df = pd.DataFrame.from_dict(rule_supports, orient='index').fillna(0)
supports_df.reset_index(inplace=True)
supports_df.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
from collections import Counter

# Initialize a Counter object to hold frequencies of antecedents and consequents
item_frequencies = Counter()

for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        # Aggregate antecedents
        for items in df['antecedents']:
            item_frequencies.update(items)
        # Aggregate consequents
        for items in df['consequents']:
            item_frequencies.update(items)

# Convert the Counter object to a DataFrame for easier manipulation and visualization

# Create a DataFrame from the Counter object
freq_df = pd.DataFrame(item_frequencies.items(), columns=['Item', 'Frequency']).sort_values(by='Frequency', ascending=False)

# Ensure the 'Item' column is of type string for later operations
freq_df['Item'] = freq_df['Item'].astype(str)


In [ ]:
rule_frequencies = {}

for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        # Iterate through each row to construct sorted rule identifiers
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            if rule_identifier not in rule_frequencies:
                rule_frequencies[rule_identifier] = {shop: 1}
            else:
                if shop in rule_frequencies[rule_identifier]:
                    rule_frequencies[rule_identifier][shop] += 1
                else:
                    rule_frequencies[rule_identifier][shop] = 1

# Convert the rule frequencies into a DataFrame for easier manipulation
rules_df = pd.DataFrame.from_dict(rule_frequencies, orient='index').fillna(0)
rules_df.reset_index(inplace=True)
rules_df.rename(columns={'index': 'Rule'}, inplace=True)

# Now, each rule string is sorted within itself by the numeric part of the ID.


In [ ]:
 rule_lifts = {}

for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        # Iterate through each row to extract rule lift
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            # Initialize or update the lift for the rule in the specific shop
            rule_lifts.setdefault(rule_identifier, {})[shop] = row['lift']

# Convert the rule lifts into a DataFrame for easier manipulation
lifts_df = pd.DataFrame.from_dict(rule_lifts, orient='index').fillna(0)
lifts_df.reset_index(inplace=True)
lifts_df.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
# Initialize a dictionary to store the confidence values for each rule
rule_confidences = {}

# Iterate over each shop and year to extract confidence values for each rule
for shop, years_dfs in meta_dfs.items():
    for year, df in years_dfs.items():
        # Iterate through each row to extract rule confidence
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            # Initialize or update the confidence for the rule in the specific shop
            rule_confidences.setdefault(rule_identifier, {})[shop] = row['confidence']

# Convert the rule confidences into a DataFrame for easier manipulation
confidences_df = pd.DataFrame.from_dict(rule_confidences, orient='index').fillna(0)
confidences_df.reset_index(inplace=True)
confidences_df.rename(columns={'index': 'Rule'}, inplace=True)

# Frequency

In [ ]:
# Plotting the heatmap
plt.figure(figsize=(94, 70))
sns.heatmap(rules_df.set_index('Rule'), annot=True, fmt=".0f", cmap="YlGnBu")
plt.title('Rule Frequencies Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(fontsize=50)
plt.show()


# Rule Support

In [ ]:
# Assuming supports_df is already in a suitable format for plotting
plt.figure(figsize=(60, 55))
sns.heatmap(supports_df.set_index('Rule'), annot=True, cmap="YlGnBu", fmt=".2f")
plt.title('Rule Support Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(fontsize=50)

# Save the plot as an SVG file
plt.savefig("rule_support_across_shops.svg", format='svg')

# Display the plot
plt.show()

In [ ]:
# Initialize a dictionary to store the total Support and count of occurrences for each rule
total_support_per_rule = {}
count_per_rule = {}

# Iterate over each row to calculate the total Support and count of occurrences for each rule
for _, row in supports_df.iterrows():
    rule = row['Rule']
    support_values = row.drop('Rule').values
    for shop, support in zip(supports_df.columns[1:], support_values):
        if support >= 0:  # Consider only non-negative support values
            if rule not in total_support_per_rule:
                total_support_per_rule[rule] = support
                count_per_rule[rule] = 1
            else:
                total_support_per_rule[rule] += support
                count_per_rule[rule] += 1

# Calculate the average Support for each rule
average_support_per_rule = {rule: total_support / count for rule, total_support, count in zip(total_support_per_rule.keys(), total_support_per_rule.values(), count_per_rule.values())}

# Sort the rules based on their average Support values in descending order
top_10_rules_support = sorted(average_support_per_rule.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Support
print("Top 10 rules with the highest average Support:")
for rule, avg_support in top_10_rules_support:
    total_support = total_support_per_rule.get(rule, 0)  # Get the total support or default to 0 if not present
    count = count_per_rule.get(rule, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule}, Average Support: {avg_support}, Total Support: {total_support}, Number of Occurrences: {count}")


# Rule Confidence

In [ ]:
# Assuming confidences_df is already in a suitable format for plotting
plt.figure(figsize=(60, 45))
sns.heatmap(confidences_df.set_index('Rule'), annot=True, fmt=".2f", cmap="YlGnBu")
plt.title('Rule Confidence Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(fontsize=50)

# Save the plot as an SVG file
plt.savefig("rule_confidence_across_shops.svg", format='svg')

# Display the plot
plt.show()

In [ ]:
# Initialize a dictionary to store the total Confidence and count of occurrences for each rule
total_confidence_per_rule = {}
count_per_rule = {}

# Iterate over each row to calculate the total Confidence and count of occurrences for each rule
for _, row in confidences_df.iterrows():
    rule = row['Rule']
    confidence_values = row.drop('Rule').values
    for shop, confidence in zip(confidences_df.columns[1:], confidence_values):
        if confidence >= 0:  # Consider only non-negative confidence values
            if rule not in total_confidence_per_rule:
                total_confidence_per_rule[rule] = confidence
                count_per_rule[rule] = 1
            else:
                total_confidence_per_rule[rule] += confidence
                count_per_rule[rule] += 1

# Calculate the average Confidence for each rule
average_confidence_per_rule = {rule: total_confidence / count for rule, total_confidence, count in zip(total_confidence_per_rule.keys(), total_confidence_per_rule.values(), count_per_rule.values())}

# Sort the rules based on their average Confidence values in descending order
top_10_rules_confidence = sorted(average_confidence_per_rule.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Confidence
print("Top 10 rules with the highest average Confidence:")
for rule, avg_confidence in top_10_rules_confidence:
    total_confidence = total_confidence_per_rule.get(rule, 0)  # Get the total confidence or default to 0 if not present
    count = count_per_rule.get(rule, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule}, Average Confidence: {avg_confidence}, Total Confidence: {total_confidence}, Number of Occurrences: {count}")


# Rule Lift

In [ ]:
# Plotting the heatmap
plt.figure(figsize=(72, 62))
sns.heatmap(lifts_df.set_index('Rule'), annot=True, cmap="YlGnBu", fmt=".2f")
plt.title('Rule Lift Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(fontsize=50)
plt.show()

In [ ]:
lifts_df.head()

In [ ]:
# Sort the lifts_df DataFrame by the 'Lift' column in descending order
lifts_df_new = lifts_df.drop(columns=['shop12'])

# Plotting the heatmap
plt.figure(figsize=(72, 62))
sns.heatmap(lifts_df_new.set_index('Rule'), annot=True, cmap="YlGnBu", fmt=".2f")
plt.title('Rule Lift Across Shops (Excluding Top 3 Highest Lift)')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(fontsize=50)

# Save the plot as an SVG file
svg_filename = "rule_lift_across_shops.svg"
plt.savefig(svg_filename, format='svg')

plt.show()


In [ ]:
# Initialize a dictionary to store the total Lift and count of occurrences for each rule
total_lift_per_rule = {}
count_per_rule = {}

# Iterate over each row to calculate the total Lift and count of occurrences for each rule
for _, row in lifts_df.iterrows():
    rule = row['Rule']
    lift_values = row.drop('Rule').values
    for shop, lift in zip(lifts_df.columns[1:], lift_values):
        if lift >= 0:  # Consider only non-negative lift values
            if rule not in total_lift_per_rule:
                total_lift_per_rule[rule] = lift
                count_per_rule[rule] = 1
            else:
                total_lift_per_rule[rule] += lift
                count_per_rule[rule] += 1

# Calculate the average Lift for each rule
average_lift_per_rule = {rule: total_lift / count for rule, total_lift, count in zip(total_lift_per_rule.keys(), total_lift_per_rule.values(), count_per_rule.values())}

# Sort the rules based on their average Lift values in descending order
top_10_rules_lift = sorted(average_lift_per_rule.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Lift
print("Top 10 rules with the highest average Lift:")
for rule, avg_lift in top_10_rules_lift:
    total_lift = total_lift_per_rule.get(rule, 0)  # Get the total lift or default to 0 if not present
    count = count_per_rule.get(rule, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule}, Average Lift: {avg_lift}, Total Lift: {total_lift}, Number of Occurrences: {count}")


## Finding out the top rules based on confidence, lift and support

In [ ]:
# Example of converting a list to DataFrame if necessary
if isinstance(top_10_rules_lift, list):
    top_10_rules_lift = pd.DataFrame(top_10_rules_lift, columns=['Rule', 'Metric_Value'])
if isinstance(top_10_rules_support, list):
    top_10_rules_support = pd.DataFrame(top_10_rules_support, columns=['Rule', 'Metric_Value'])
if isinstance(top_10_rules_confidence, list):
    top_10_rules_confidence = pd.DataFrame(top_10_rules_confidence, columns=['Rule', 'Metric_Value'])


In [ ]:
top_10_rules_lift = top_10_rules_lift.sort_values(by='Metric_Value', ascending=False)
top_10_rules_support = top_10_rules_support.sort_values(by='Metric_Value', ascending=False)
top_10_rules_confidence = top_10_rules_confidence.sort_values(by='Metric_Value', ascending=False)


In [ ]:
# These DataFrames should have at least two columns: 'Rule' and 'Metric_Value' (where 'Metric_Value' could be lift, support, or confidence)

# Corrected function to assign scores based on rank within the DataFrame with robust indexing
def assign_scores(df):
    num_rules = min(len(df), 10)  # To handle cases with fewer than 10 entries
    # Resetting the index of DataFrame to ensure proper alignment when assigning scores
    df = df.reset_index(drop=True)
    # Create a new column 'Score' and initialize with 0
    df['Score'] = 0
    # Assign scores within the range of existing rows
    for i in range(num_rules):
        df.at[i, 'Score'] = 10 - i
    return df[['Rule', 'Score']]


# Apply the scoring function to each DataFrame
scores_lift = assign_scores(top_10_rules_lift)
scores_support = assign_scores(top_10_rules_support)
scores_confidence = assign_scores(top_10_rules_confidence)

# Combine all the scores into one DataFrame
combined_scores = pd.merge(scores_lift, scores_support, on='Rule', how='outer', suffixes=('_lift', '_support'))
combined_scores = pd.merge(combined_scores, scores_confidence, on='Rule', how='outer')
combined_scores.fillna(0, inplace=True)  # Replace NaN scores (for rules not in top 10 in some metric) with 0

# Calculate the total score for each rule
combined_scores['Total_Score'] = combined_scores['Score_lift'] + combined_scores['Score_support'] + combined_scores['Score']

# Sort by total score in descending order to find the highest ranked rules
combined_scores.sort_values(by='Total_Score', ascending=False, inplace=True)



In [ ]:
combined_scores.head(5)

In [ ]:
# Define the directory and file path
directory = os.path.expanduser("~/Desktop/Master Project newest/data")
file_path = os.path.join(directory, "combined_scores.csv")

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the DataFrame to a CSV file
combined_scores.to_csv(file_path, index=False)

print(f"DataFrame saved to {file_path}")

# Top 5 for 2023

In [ ]:
def order_id_frozensets(df, columns=['antecedents', 'consequents']):
    """
    Orders the ID frozensets within specified columns of a DataFrame.

    Args:
    df (DataFrame): The DataFrame to process.
    columns (list of str): List of column names to order frozensets in.
    """
    for col in columns:
        # Convert frozenset to sorted list, then back to frozenset
        df[col] = df[col].apply(lambda x: frozenset(sorted(x)))
    return df

# Apply the function to each DataFrame in meta_dfs
for shop, years_dfs in meta_dfs_2023.items():
    for year, df in years_dfs.items():
        meta_dfs_2023[shop][year] = order_id_frozensets(df, columns=['antecedents', 'consequents'])

# Now, meta_dfs has been updated with ordered frozensets in the specified columns.


In [ ]:
# Assuming meta_dfs is your dictionary containing all the shop dataframes
all_data_2023 = []
for shop, years in meta_dfs_2023.items():
    for year, df in years.items():
        df['shop'] = shop
        df['year'] = year
        all_data_2023.append(df)

# Concatenate all the dataframes into one
combined_data_2023 = pd.concat(all_data_2023, ignore_index=True)

# Create 'rule' column if it doesn't exist
combined_data_2023['rule'] = combined_data_2023['LHS'] + ' → ' + combined_data_2023['RHS']

In [ ]:
# Initialize a dictionary to store the support values for each rule
rule_supports_2023 = {}

# Iterate over each shop and year to extract support values for each rule
for shop_2023, years_dfs_2023 in meta_dfs.items():
    for year_2023, df_2023 in years_dfs_2023.items():
        # Iterate through each row to extract rule support
        for _, row_2023 in df_2023.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents_2023 = sorted(list(row_2023['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents_2023 = sorted(list(row_2023['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_2023 = f"{tuple(sorted_antecedents_2023)} -> {tuple(sorted_consequents_2023)}"
            
            # Initialize or update the support for the rule in the specific shop
            rule_supports_2023.setdefault(rule_identifier_2023, {})[shop_2023] = row_2023['support']

# Convert the rule supports into a DataFrame for easier manipulation
supports_df_2023 = pd.DataFrame.from_dict(rule_supports_2023, orient='index').fillna(0)
supports_df_2023.reset_index(inplace=True)
supports_df_2023.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
rule_lifts_2023 = {}

for shop_2023, years_dfs_2023 in meta_dfs.items():
    for year_2023, df_2023 in years_dfs_2023.items():
        # Iterate through each row to extract rule lift
        for _, row_2023 in df_2023.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents_2023 = sorted(list(row_2023['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents_2023 = sorted(list(row_2023['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_2023 = f"{tuple(sorted_antecedents_2023)} -> {tuple(sorted_consequents_2023)}"
            
            # Initialize or update the lift for the rule in the specific shop
            rule_lifts_2023.setdefault(rule_identifier_2023, {})[shop_2023] = row_2023['lift']

# Convert the rule lifts into a DataFrame for easier manipulation
lifts_df_2023 = pd.DataFrame.from_dict(rule_lifts_2023, orient='index').fillna(0)
lifts_df_2023.reset_index(inplace=True)
lifts_df_2023.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
# Initialize a dictionary to store the confidence values for each rule
rule_confidences_2023 = {}

# Iterate over each shop and year to extract confidence values for each rule
for shop_2023, years_dfs_2023 in meta_dfs.items():
    for year_2023, df_2023 in years_dfs_2023.items():
        # Iterate through each row to extract rule confidence
        for _, row_2023 in df_2023.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents_2023 = sorted(list(row_2023['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents_2023 = sorted(list(row_2023['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_2023 = f"{tuple(sorted_antecedents_2023)} -> {tuple(sorted_consequents_2023)}"
            
            # Initialize or update the confidence for the rule in the specific shop
            rule_confidences_2023.setdefault(rule_identifier_2023, {})[shop_2023] = row_2023['confidence']

# Convert the rule confidences into a DataFrame for easier manipulation
confidences_df_2023 = pd.DataFrame.from_dict(rule_confidences_2023, orient='index').fillna(0)
confidences_df_2023.reset_index(inplace=True)
confidences_df_2023.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
# Initialize a dictionary to store the total Support and count of occurrences for each rule
total_support_per_rule_2023 = {}
count_per_rule_2023 = {}

# Iterate over each row to calculate the total Support and count of occurrences for each rule
for _, row_2023 in supports_df_2023.iterrows():
    rule_2023 = row_2023['Rule']
    support_values_2023 = row_2023.drop('Rule').values
    for shop_2023, support_2023 in zip(supports_df_2023.columns[1:], support_values_2023):
        if support_2023 >= 0:  # Consider only non-negative support values
            if rule_2023 not in total_support_per_rule_2023:
                total_support_per_rule_2023[rule_2023] = support_2023
                count_per_rule_2023[rule_2023] = 1
            else:
                total_support_per_rule_2023[rule_2023] += support_2023
                count_per_rule_2023[rule_2023] += 1

# Calculate the average Support for each rule
average_support_per_rule_2023 = {rule_2023: total_support_2023 / count_2023 for rule_2023, total_support_2023, count_2023 in zip(total_support_per_rule_2023.keys(), total_support_per_rule_2023.values(), count_per_rule_2023.values())}

# Sort the rules based on their average Support values in descending order
top_10_rules_support_2023 = sorted(average_support_per_rule_2023.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Support
print("Top 10 rules with the highest average Support:")
for rule_2023, avg_support_2023 in top_10_rules_support_2023:
    total_support_2023 = total_support_per_rule_2023.get(rule_2023, 0)  # Get the total support or default to 0 if not present
    count_2023 = count_per_rule_2023.get(rule_2023, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule_2023}, Average Support: {avg_support_2023}, Total Support: {total_support_2023}, Number of Occurrences: {count_2023}")


In [ ]:
# Initialize a dictionary to store the total Confidence and count of occurrences for each rule
total_confidence_per_rule_2023 = {}
count_per_rule_2023 = {}

# Iterate over each row to calculate the total Confidence and count of occurrences for each rule
for _, row_2023 in confidences_df_2023.iterrows():
    rule_2023 = row_2023['Rule']
    confidence_values_2023 = row_2023.drop('Rule').values
    for shop_2023, confidence_2023 in zip(confidences_df_2023.columns[1:], confidence_values_2023):
        if confidence_2023 >= 0:  # Consider only non-negative confidence values
            if rule_2023 not in total_confidence_per_rule_2023:
                total_confidence_per_rule_2023[rule_2023] = confidence_2023
                count_per_rule_2023[rule_2023] = 1
            else:
                total_confidence_per_rule_2023[rule_2023] += confidence_2023
                count_per_rule_2023[rule_2023] += 1

# Calculate the average Confidence for each rule
average_confidence_per_rule_2023 = {rule_2023: total_confidence_2023 / count_2023 for rule_2023, total_confidence_2023, count_2023 in zip(total_confidence_per_rule_2023.keys(), total_confidence_per_rule_2023.values(), count_per_rule_2023.values())}

# Sort the rules based on their average Confidence values in descending order
top_10_rules_confidence_2023 = sorted(average_confidence_per_rule_2023.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Confidence
print("Top 10 rules with the highest average Confidence:")
for rule_2023, avg_confidence_2023 in top_10_rules_confidence_2023:
    total_confidence_2023 = total_confidence_per_rule_2023.get(rule_2023, 0)  # Get the total confidence or default to 0 if not present
    count_2023 = count_per_rule_2023.get(rule_2023, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule_2023}, Average Confidence: {avg_confidence_2023}, Total Confidence: {total_confidence_2023}, Number of Occurrences: {count_2023}")


In [ ]:
# Initialize a dictionary to store the total Lift and count of occurrences for each rule
total_lift_per_rule_2023 = {}
count_per_rule_2023 = {}

# Iterate over each row to calculate the total Lift and count of occurrences for each rule
for _, row_2023 in lifts_df_2023.iterrows():
    rule_2023 = row_2023['Rule']
    lift_values_2023 = row_2023.drop('Rule').values
    for shop_2023, lift_2023 in zip(lifts_df_2023.columns[1:], lift_values_2023):
        if lift_2023 >= 0:  # Consider only non-negative lift values
            if rule_2023 not in total_lift_per_rule_2023:
                total_lift_per_rule_2023[rule_2023] = lift_2023
                count_per_rule_2023[rule_2023] = 1
            else:
                total_lift_per_rule_2023[rule_2023] += lift_2023
                count_per_rule_2023[rule_2023] += 1

# Calculate the average Lift for each rule
average_lift_per_rule_2023 = {rule_2023: total_lift_2023 / count_2023 for rule_2023, total_lift_2023, count_2023 in zip(total_lift_per_rule_2023.keys(), total_lift_per_rule_2023.values(), count_per_rule_2023.values())}

# Sort the rules based on their average Lift values in descending order
top_10_rules_lift_2023 = sorted(average_lift_per_rule_2023.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 rules with the highest average Lift
print("Top 10 rules with the highest average Lift:")
for rule_2023, avg_lift_2023 in top_10_rules_lift_2023:
    total_lift_2023 = total_lift_per_rule_2023.get(rule_2023, 0)  # Get the total lift or default to 0 if not present
    count_2023 = count_per_rule_2023.get(rule_2023, 0)  # Get the count or default to 0 if not present
    print(f"Rule: {rule_2023}, Average Lift: {avg_lift_2023}, Total Lift: {total_lift_2023}, Number of Occurrences: {count_2023}")


## Finding out the top rules based on confidence, lift and support

In [ ]:
# Example of converting a list to DataFrame if necessary
if isinstance(top_10_rules_lift_2023, list):
    top_10_rules_lift_2023 = pd.DataFrame(top_10_rules_lift_2023, columns=['Rule_2023', 'Metric_Value_2023'])
if isinstance(top_10_rules_support_2023, list):
    top_10_rules_support_2023 = pd.DataFrame(top_10_rules_support_2023, columns=['Rule_2023', 'Metric_Value_2023'])
if isinstance(top_10_rules_confidence_2023, list):
    top_10_rules_confidence_2023 = pd.DataFrame(top_10_rules_confidence_2023, columns=['Rule_2023', 'Metric_Value_2023'])


In [ ]:
top_10_rules_lift_2023 = top_10_rules_lift_2023.sort_values(by='Metric_Value_2023', ascending=False)
top_10_rules_support_2023 = top_10_rules_support_2023.sort_values(by='Metric_Value_2023', ascending=False)
top_10_rules_confidence_2023 = top_10_rules_confidence_2023.sort_values(by='Metric_Value_2023', ascending=False)



In [ ]:
# These DataFrames should have at least two columns: 'Rule' and 'Metric_Value' (where 'Metric_Value' could be lift, support, or confidence)

# Corrected function to assign scores based on rank within the DataFrame with robust indexing
def assign_scores_2023(df_2023):
    num_rules_2023 = min(len(df_2023), 10)  # To handle cases with fewer than 10 entries
    # Resetting the index of DataFrame to ensure proper alignment when assigning scores
    df_2023 = df_2023.reset_index(drop=True)
    # Create a new column 'Score' and initialize with 0
    df_2023['Score_2023'] = 0
    # Assign scores within the range of existing rows
    for i in range(num_rules_2023):
        df_2023.at[i, 'Score_2023'] = 10 - i
    return df_2023[['Rule_2023', 'Score_2023']]

# Note: The function now ensures the index is reset and uses the .at method for precise assignment.
# The other code sections for merging and totaling scores should be reused with this updated function definition.
# Uncomment when ready to implement in your local environment.

# Apply the scoring function to each DataFrame
scores_lift_2023 = assign_scores_2023(top_10_rules_lift_2023)
scores_support_2023 = assign_scores_2023(top_10_rules_support_2023)
scores_confidence_2023 = assign_scores_2023(top_10_rules_confidence_2023)

# Combine all the scores into one DataFrame
combined_scores_2023 = pd.merge(scores_lift_2023, scores_support_2023, on='Rule_2023', how='outer', suffixes=('_lift_2023', '_support_2023'))
combined_scores_2023 = pd.merge(combined_scores_2023, scores_confidence_2023, on='Rule_2023', how='outer')
combined_scores_2023.fillna(0, inplace=True)  # Replace NaN scores (for rules not in top 10 in some metric) with 0

# Correct the calculation of the total score
combined_scores_2023['Total_Score_2023'] = combined_scores_2023['Score_2023_lift_2023'] + combined_scores_2023['Score_2023_support_2023'] + combined_scores_2023['Score_2023']


# Sort by total score in descending order to find the highest ranked rules
combined_scores_2023.sort_values(by='Total_Score_2023', ascending=False, inplace=True)

# Uncomment to view the DataFrame with the top rules based on their combined scores


In [ ]:
combined_scores_2023.head(5)

# Meta Analysis, Time Series Shop 1 Search 22

In [ ]:
# Group data by 'rule', 'shop', and 'year'
rule_metrics_time = combined_data.groupby(['rule', 'shop', 'year']).agg({
    'support': 'mean',
    'confidence': 'mean',
    'lift': 'mean'
}).reset_index()

# Calculate the maximum year for each shop and convert it into a DataFrame for merging
max_year_per_shop = rule_metrics_time.groupby('shop')['year'].max().reset_index()
max_year_per_shop.rename(columns={'year': 'year_max'}, inplace=True)

# Merge the maximum year information back into the rule metrics
rule_first_last_year = rule_metrics_time.groupby(['rule', 'shop']).agg(
    first_year=('year', 'min'),
    last_year=('year', 'max')
).reset_index()

rule_first_last_year = rule_first_last_year.merge(max_year_per_shop, on='shop', how='left')
rule_first_last_year['is_current'] = rule_first_last_year['last_year'] == rule_first_last_year['year_max']

# Filter to analyze further or visualize trends
print(rule_first_last_year)


In [ ]:
# Filter out active rules
active_rules = rule_first_last_year[rule_first_last_year['is_current']]

# Printing active rules for verification
print(active_rules)


In [ ]:
# Merge active rules with original metrics to get the full data for these rules
active_rule_metrics = active_rules.merge(rule_metrics_time, on=['rule', 'shop'], how='left')

# Aggregate to get the mean confidence for each rule across all years it was active
active_rule_confidence = active_rule_metrics.groupby(['rule', 'shop']).agg({
    'confidence': 'mean'
}).reset_index()

# Aggregate to get the mean confidence for each rule across all years it was active
active_rule_support = active_rule_metrics.groupby(['rule', 'shop']).agg({
    'support': 'mean'
}).reset_index()


# Aggregate to get the mean confidence for each rule across all years it was active
active_rule_lift = active_rule_metrics.groupby(['rule', 'shop']).agg({
    'lift': 'mean'
}).reset_index()




In [ ]:
# Sorting the results by confidence for better visualization
active_rule_confidence_sorted = active_rule_confidence.sort_values(by='confidence', ascending=False)

# Sorting the results by Lift for better visualization
active_rule_lift_sorted = active_rule_lift.sort_values(by='lift', ascending=False)

# Sorting the results by Support for better visualization
active_rule_support_sorted = active_rule_support.sort_values(by='support', ascending=False)

In [ ]:
active_rule_lift_sorted.head()

In [ ]:
# Merge the DataFrames based on the 'Rule' column
combined_df = pd.merge(active_rule_lift_sorted, active_rule_support_sorted, on='rule')
combined_df = pd.merge(combined_df, active_rule_confidence_sorted, on='rule')

# Calculate a combined score based on lift, support, and confidence
# You can define your own formula for combining the scores based on your business requirements
# For example, you can simply sum the three metrics or assign different weights to each metric
combined_df['Combined_Score'] = combined_df['lift'] + combined_df['support'] + combined_df['confidence']

# Sort the DataFrame by the combined score in descending order
combined_df = combined_df.sort_values(by='Combined_Score', ascending=False)


In [ ]:
# Setting up the plot
plt.figure(figsize=(10, 30))
sns.barplot(x='confidence', y='rule', data=active_rule_confidence_sorted, hue='shop', dodge=False)
plt.title('Mean Confidence of Active Rules by Shop')
plt.xlabel('Mean Confidence')
plt.ylabel('Rule')
plt.legend(title='Shop')
plt.show()


In [ ]:
# Setting up the plot
plt.figure(figsize=(10, 30))
sns.barplot(x='lift', y='rule', data=active_rule_lift_sorted, hue='shop', dodge=False)
plt.title('Mean Lift of Active Rules by Shop')
plt.xlabel('Mean Lift')
plt.ylabel('Rule')
plt.legend(title='Shop')
plt.show()


In [ ]:
# Setting up the plot
plt.figure(figsize=(10, 30))
sns.barplot(x='support', y='rule', data=active_rule_support_sorted, hue='shop', dodge=False)
plt.title('Mean Support of Active Rules by Shop')
plt.xlabel('Mean Support')
plt.ylabel('Rule')
plt.legend(title='Shop')
plt.show()


In [ ]:
# Normalize the metrics (support, confidence, and lift) to be between 0 and 1
combined_df['normalized_support'] = (combined_df['support'] - combined_df['support'].min()) / (combined_df['support'].max() - combined_df['support'].min())
combined_df['normalized_confidence'] = (combined_df['confidence'] - combined_df['confidence'].min()) / (combined_df['confidence'].max() - combined_df['confidence'].min())
combined_df['normalized_lift'] = (combined_df['lift'] - combined_df['lift'].min()) / (combined_df['lift'].max() - combined_df['lift'].min())

# Calculate the combined score for each rule using normalized metrics
combined_df['combined_score_normalized'] = combined_df['normalized_support'] + combined_df['normalized_confidence'] + combined_df['normalized_lift']

# Sorting the results by the combined score for better visualization
combined_df_sorted_normalized = combined_df.sort_values(by='combined_score_normalized', ascending=False)

# Setting up the plot
plt.figure(figsize=(10, 30))
sns.barplot(x='combined_score_normalized', y='rule', data=combined_df_sorted_normalized, hue='shop', dodge=False)
plt.title('Combined Score of Active Rules by Shop (Normalized)')
plt.xlabel('Combined Score (Normalized)')
plt.ylabel('Rule')
plt.legend(title='Shop')
plt.show()


# Ideas

From here we can observe, which active rule has the highest value in the different metrics. This leads to different managerial implications:

### Strategic Implications of Association Rule Metrics

**1. Lift:**
Lift is a metric that measures how much more often the antecedent and consequent of a rule occur together than we would expect if they were statistically independent. A rule with a high lift value indicates a strong, positive association between the antecedent and consequent. **Strategic implications:** Rules with high lift are particularly valuable for cross-selling strategies because they identify items that strongly influence the purchase of other items. For instance, a high lift between bread and butter suggests that customers who buy bread are much more likely to buy butter than the average shopper. Promotions can be structured to capitalize on these strong associations, potentially by bundling items together or positioning them close to each other in store layouts or online categories.

**2. Support:**
Support measures the proportion of transactions that include both the antecedent and the consequent. It gives an idea of how frequently a rule is applicable in the dataset. **Strategic implications:** A rule with high support impacts a larger segment of the transactions, making it crucial for mass-market strategies. High-support rules are robust bases for making general stocking and marketing decisions, as they affect a significant portion of customer transactions. For example, if milk and cereal have high support, this rule can guide store inventory decisions to ensure these popular items are always in stock, optimizing turnover rates.

**3. Confidence:**
Confidence assesses the reliability of the inference made by the rule, calculated as the probability of seeing the consequent given the antecedent. **Strategic implications:** High confidence in a rule indicates that the presence of the antecedent significantly increases the likelihood of the consequent’s presence in transactions. This metric is critical for targeting marketing campaigns and personalizing recommendations. For instance, if there is high confidence that customers who purchase running shoes also buy sports drinks, targeted offers can be made to these customers, increasing the probability of additional purchases.

### How Strategic Implications Differ by Metric:

- **Lift vs. Support vs. Confidence:** While support indicates the usefulness and productivity of a rule across all transactions, lift provides insight into the strength of an association independent of the prevalence of the outcome, making it ideal for uncovering hidden relationships in data. Confidence, on the other hand, helps in understanding the likelihood or reliability of predicting the consequent, thus being directly actionable for personalized marketing strategies.
- **Decision Making:** High lift values can identify niche but potentially profitable associations that might be overlooked when only considering support. In contrast, high confidence rules can be misleading if based on rare item combinations, unless verified with substantial support or lift. Therefore, combining these metrics provides a balanced view: lift to find valuable associations, support to assess broad impact, and confidence to ensure predictability in targeted actions.
- **Resource Allocation:** Understanding these metrics helps in prioritizing resource allocation in promotional activities, inventory management, and personalized marketing. For example, high lift and confidence rules might direct more aggressive cross-promotion tactics, whereas high support might influence broader stock level decisions.

By strategically analyzing these metrics, businesses can tailor their operational and marketing strategies more effectively, ensuring that they not only meet the general demand but also exploit specific, strong item associations to enhance customer satisfaction and increase sales.

In [ ]:
# Ensure you have the initial data setup and grouped correctly, like this:
# rule_metrics_time = combined_data.groupby(['rule', 'shop', 'year']).agg({
#     'support': 'mean',
#     'confidence': 'mean',
#     'lift': 'mean'
# }).reset_index()

# Prepare the growth_decay DataFrame
growth_decay = rule_first_last_year.copy()

# Calculate growth and decay rates for support, confidence, and lift
growth_decay['support_change'] = growth_decay.apply(
    lambda x: (
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['last_year'])]['support'].values[0] -
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['support'].values[0]
    ) / rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['support'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

growth_decay['confidence_change'] = growth_decay.apply(
    lambda x: (
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['last_year'])]['confidence'].values[0] -
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['confidence'].values[0]
    ) / rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['confidence'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

growth_decay['lift_change'] = growth_decay.apply(
    lambda x: (
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['last_year'])]['lift'].values[0] -
        rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['lift'].values[0]
    ) / rule_metrics_time[(rule_metrics_time['rule'] == x['rule']) & (rule_metrics_time['shop'] == x['shop']) & (rule_metrics_time['year'] == x['first_year'])]['lift'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

# Now, filter for significant changes
significant_confidence_changes = growth_decay[growth_decay['confidence_change'].abs() > 0.1]
significant_lift_changes = growth_decay[growth_decay['lift_change'].abs() > 0.1]
significant_support_changes = growth_decay[growth_decay['support_change'].abs() > 0.1]

In [ ]:
# Sort the DataFrame by 'confidence_change' in descending order to show the most significant changes first
sorted_significant_changes = significant_confidence_changes.sort_values(by='confidence_change', ascending=False)

# Now, create the bar plot with the sorted data
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='confidence_change', y='rule', data=sorted_significant_changes, hue='shop', dodge=False)
plt.title('Significant Confidence Changes in Rules')
plt.xlabel('Confidence Change Rate')
plt.ylabel('Rule')
plt.grid(True)

# Save the plot as an SVG file
plt.savefig("significant_confidence_changes.svg", format='svg', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Sort the DataFrame by 'lift_change' in descending order to show the most significant changes first
sorted_significant_changes = significant_lift_changes.sort_values(by='lift_change', ascending=False)

# Create the bar plot with the sorted data
plt.figure(figsize=(12, 8))  # Increase figure size for better visibility
bar_plot = sns.barplot(x='lift_change', y='rule', data=sorted_significant_changes, hue='shop', dodge=False)
plt.title('Significant Lift Changes in Rules')
plt.xlabel('Lift Change Rate')
plt.ylabel('Rule')
plt.grid(True)

# Adjust layout to make sure everything fits
plt.tight_layout()

# Save the plot as an SVG file
plt.savefig("significant_lift_changes.svg", format='svg', bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
# Create the bar plot with sorted data
plt.figure(figsize=(10, 6))
sorted_significant_changes = significant_support_changes.sort_values(by='support_change', ascending=False)
sns.barplot(x='support_change', y='rule', data=sorted_significant_changes, hue='shop', dodge=False)
plt.title('Significant Support Changes in Rules (Sorted)')
plt.xlabel('Support Change Rate')
plt.ylabel('Rule')
plt.grid(True)

# Save the plot as an SVG file
plt.savefig("significant_support_changes.svg", format='svg', bbox_inches='tight')

# Display the plot
plt.show()

# Meta Analysis, Multi dimensional Scaling, Search 25

In [ ]:
# Assuming combined_data is already defined and has 'shop', 'antecedents', and 'consequents' columns

# Create a custom rule representation using tuples of sorted antecedents and consequents
def create_rule(antecedents, consequents):
    return (tuple(sorted(antecedents)), tuple(sorted(consequents)))

# Create a dictionary to accumulate rules for each shop
shops = combined_data['shop'].unique()
shop_rules = {shop: set() for shop in shops}

for _, row in combined_data.iterrows():
    rule = create_rule(row['antecedents'], row['consequents'])
    shop_rules[row['shop']].add(rule)

# Check rules for Shop11 in shop_rules
print(f"Rules for Shop11 in shop_rules:\n{shop_rules['shop11']}")
print(f"Number of unique rules for Shop11 in shop_rules: {len(shop_rules['shop11'])}")

# Create a binary matrix where each row represents a shop and each column a unique rule
all_rules = list(set.union(*shop_rules.values()))  # Convert the set to a list
rule_matrix = pd.DataFrame(0, index=shops, columns=all_rules, dtype=int)

for shop in shops:
    for rule in shop_rules[shop]:
        rule_matrix.at[shop, rule] = 1

# Convert DataFrame to numpy array for distance calculation
rule_matrix_np = rule_matrix.to_numpy()

# Compute the similarity matrix using Jaccard similarity
similarity_matrix = 1 - pairwise_distances(rule_matrix_np, metric='jaccard')

# Apply MDS
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
shop_positions = mds.fit_transform(similarity_matrix)  # Already 1 - similarity_matrix gives dissimilarity

# Prepare results for plotting
shop_positions_df = pd.DataFrame(shop_positions, index=shops, columns=['x', 'y'])

# Visualize the results
plt.figure(figsize=(15, 10))
for shop in shops:
    plt.scatter(shop_positions_df.at[shop, 'x'], shop_positions_df.at[shop, 'y'], label=shop)
    plt.text(shop_positions_df.at[shop, 'x'] + 0.001, shop_positions_df.at[shop, 'y'] + 0.001, shop)  # Slight offset for label

plt.title('2D MDS of Shop Similarities in Association Rules')
plt.xlabel('MDS Dimension 1')
plt.ylabel('MDS Dimension 2')
plt.legend(title='Shops')
plt.grid(True)
plt.show()


How can it be, that shop 7,8,9 have similarities in the heatmap, are in the same cluster, but look so differently in the MDS? 

# Meta Analysis, Clustering to see differences, Search 26

In [ ]:
# Suppress specific DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message="is_sparse is deprecated")


# Example DataFrame structure
data = {
    'shop': combined_data['shop'],
    'rule': combined_data['antecedents'].astype(str) + '->' + combined_data['consequents'].astype(str),
    'confidence': combined_data['confidence']
}

df = pd.DataFrame(data)

# Pivot table to transform data into a [shops x rules] matrix, filling missing rules with zero
feature_matrix = df.pivot_table(index='shop', columns='rule', values='confidence', fill_value=0)

# Normalize the feature matrix
scaler = MinMaxScaler()
feature_matrix_normalized = scaler.fit_transform(feature_matrix)
feature_matrix_normalized = pd.DataFrame(feature_matrix_normalized, index=feature_matrix.index, columns=feature_matrix.columns)


In [ ]:
feature_matrix.head()

In [ ]:
df.head()

In [ ]:
# Determine the optimal number of clusters using the elbow method
sse = {}
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(feature_matrix_normalized)
    sse[k] = kmeans.inertia_  # Sum of squared distances of samples to their closest cluster center

plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method For Optimal k")
plt.show()

# Assume the elbow is found at k=4
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(feature_matrix_normalized)

# Add cluster information back to the original DataFrame
feature_matrix['Cluster'] = clusters


In [ ]:
# Assuming the rest of the preprocessing and clustering code has been run, starting from PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(feature_matrix_normalized)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Cluster'] = clusters

# Assuming 'pca_df' already has PCA results and cluster labels
# Let's add shop identifiers to the DataFrame for easy access
pca_df['Shop'] = feature_matrix.index  # This assumes the index of your feature matrix has shop identifiers

# Now plotting with annotations
plt.figure(figsize=(10, 8))
plot = sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=pca_df, palette='viridis', s=100, alpha=0.7)

# Adding text annotations for each point
for i in range(pca_df.shape[0]):
    plt.text(x=pca_df.PC1[i] + 0.02,  # x-coordinate position for text
             y=pca_df.PC2[i] + 0.02,  # y-coordinate position for text
             s=pca_df.Shop[i],  # text label
             fontdict=dict(color='black', size=10),
            )

plt.title('PCA Cluster Plot of Shops with Shop Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)

# Save the plot as SVG
svg_filename = "pca_cluster_plot.svg"
plt.savefig(svg_filename, format='svg')
plt.show()

print(f"Plot saved as {svg_filename}")

1. Data Attributes
The primary attributes used for clustering were based on association rules, specifically:

Rules: Each unique combination of antecedents (items that lead to a purchase) and consequents (items that are purchased as a result) formed a rule.
Confidence: The strength of each rule, which measures the reliability of the rule, was used as the key attribute. Confidence is defined as the probability of seeing the consequent items given the antecedent items have been purchased.

2. Construction of the Feature Matrix
The feature matrix was constructed as follows:

Rows: Each row represented a shop.

Columns: Each column represented a unique association rule derived from the combined data of all shops. The uniqueness of a rule was based on the specific combination of items in the antecedents and consequents.

Values: The value in each cell of the matrix was the confidence level of the rule for that shop. If a shop did not have a particular rule present in its transactions, the confidence was set to zero.

3. Data Preparation for Clustering
Normalization: The feature matrix was normalized to ensure that each rule was equally weighted during the clustering process. Normalization adjusted the confidence values so that they were scaled between 0 and 1 across the dataset. This step is crucial because it prevents rules with naturally higher confidence levels from disproportionately influencing the clustering.

4. Clustering Algorithm
K-Means Clustering: This algorithm was chosen for its efficiency and effectiveness in grouping data into clusters that minimize the variance within each cluster. The number of clusters (k) was determined using the elbow method, which identifies a point where adding more clusters does not significantly improve the within-cluster sum of squares (SSE).

5. Dimensionality Reduction for Visualization
PCA (Principal Component Analysis): Though not directly involved in forming the clusters, PCA was used post-clustering to reduce the high-dimensional feature space into 2 dimensions. This reduction allowed for easy visualization of the clusters and helped interpret how shops are grouped based on the rules' presence and strength.

Conclusion

The clusters were formed based on how similarly shops behaved concerning the association rules prevalent in their transactions. By using confidence as the clustering attribute, the analysis not only considered which rules were common but also how significant these rules were in each shop's context. This approach provided a nuanced view that combined both the qualitative aspect (which rules are present) and the quantitative aspect (how strong these rules are), leading to a comprehensive grouping based on purchasing patterns. This method is particularly useful for understanding market dynamics, tailoring marketing strategies, and optimizing inventory management based on consumer behavior similarities across shops.

# Meta Analysis, Hierarchical Clustering, MDS to see differences, Search 27

In [ ]:
# Assuming 'rule_matrix' is already prepared and contains binary rule presence data

# Normalize rule_matrix for hierarchical clustering
scaler = MinMaxScaler()
rule_matrix_scaled = scaler.fit_transform(rule_matrix)

# Hierarchical Clustering
hclust = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
hclusters = hclust.fit_predict(rule_matrix_scaled)


In [ ]:
# Generate the linkage matrix
Z = sch_linkage(rule_matrix_scaled, method='ward')

# Plotting the dendrogram
plt.figure(figsize=(15, 10))
dendrogram(Z, labels=shops, leaf_rotation=90, leaf_font_size=12, color_threshold=0)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Shop')
plt.ylabel('Distance')
plt.show()


In [ ]:
# Apply MDS based on the Jaccard dissimilarity
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
shop_positions = mds.fit_transform(1 - similarity_matrix)  # using 1 - similarity_matrix for dissimilarity

# Prepare results for plotting
shop_positions_df = pd.DataFrame(shop_positions, columns=['x', 'y'], index=shops)
shop_positions_df['MDS Cluster'] = hclusters  # add hierarchical clusters for coloring in MDS


In [ ]:
# Plotting the MDS results
plt.figure(figsize=(15, 10))
scatter = plt.scatter(shop_positions_df['x'], shop_positions_df['y'], c=shop_positions_df['MDS Cluster'], cmap='viridis')
plt.title('2D MDS of Shop Similarities in Association Rules with Hierarchical Clustering')
plt.xlabel('MDS Dimension 1')
plt.ylabel('MDS Dimension 2')
plt.colorbar(scatter, label='Cluster Group')
for idx, row in shop_positions_df.iterrows():
    plt.text(row['x'], row['y'], s=idx, fontdict=dict(color='red', size=12))
plt.grid(True)
plt.show()


In [ ]:
# Example: Defining a list of shop identifiers
sorted_shops = ['shop1', 'shop11', 'shop12', 'shop13','shop3', 'shop4', 'shop5', 'shop6', 'shop7', 'shop8', 'shop9']
print("Original order of sorted_shops:")
print(sorted_shops)

# Function to extract numbers from shop identifiers
def extract_number(text):
    num = re.search(r'\d+', text)
    return int(num.group()) if num else None

# Sort the list by extracting numbers and sorting based on them
sorted_shops = sorted(sorted_shops, key=extract_number)

print("\nNumerically sorted sorted_shops:")
print(sorted_shops)


In [ ]:
pca_df.head()

In [ ]:
# Convert hclusters to a pandas Series with shops as the index
hclusters_series = pd.Series(hclusters, index=sorted_shops)

# Reindex hclusters_series to match pca_df
# This ensures that cluster labels are aligned with the PCA data
hclusters_aligned = hclusters_series.reindex(pca_df['Shop']).values

# Assign the correctly ordered clusters to pca_df
pca_df['Cluster'] = hclusters_aligned

# Now plotting with annotations
plt.figure(figsize=(10, 8))
plot = sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=pca_df, palette='viridis', s=100, alpha=0.7)

# Adding text annotations for each point
for i in range(pca_df.shape[0]):
    plt.text(x=pca_df.PC1[i] + 0.02,  # x-coordinate position for text
             y=pca_df.PC2[i] + 0.02,  # y-coordinate position for text
             s=pca_df.Shop[i],  # text label
             fontdict=dict(color='black', size=10),
            )

plt.title('PCA Hierarchical Cluster Plot of Shops with Shop Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


In [ ]:
# Create the network graph including all possible edges, with edge width varying by similarity
G = nx.Graph()

# Add all nodes for each shop
for shop in shops:
    G.add_node(shop)

# Connect all shops with edges weighted by similarity
for i, shop_i in enumerate(shops):
    for j, shop_j in enumerate(shops):
        if i < j:  # Avoid self-loops and duplicate edges
            similarity = similarity_matrix[i, j]
            # Use a continuous scale for edge width
            G.add_edge(shop_i, shop_j, weight=similarity)

# Calculate the number of rules per shop for node sizes
num_rules_per_shop = {shop: len(rules) for shop, rules in shop_rules.items()}

# Normalize the values to use for node size
sizes = [num_rules_per_shop[shop] for shop in G.nodes()]
max_size = max(sizes)
min_size = min(sizes)
node_sizes = [(size - min_size) / (max_size - min_size) * 1000 + 100 for size in sizes]  # Scale between 100 and 1100

# Use the 'MDS Cluster' for coloring nodes
cluster_colors = shop_positions_df['MDS Cluster']

# If the cluster numbers are not sequential starting from 0, map them to a sequential range
unique_clusters = sorted(cluster_colors.unique())
cluster_color_mapping = {cluster: i for i, cluster in enumerate(unique_clusters)}
sequential_cluster_colors = cluster_colors.map(cluster_color_mapping)

# Assuming G and all related variables are already defined as you have done.

# Generate a color palette with seaborn and map the clusters to colors using viridis
palette = sns.color_palette('viridis', len(unique_clusters))
node_colors = [palette[color] for color in sequential_cluster_colors]

# Draw the network with the updated color palette
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42)  # Positions should be consistent with previous plot

# Draw nodes with sizes based on the number of rules and color based on clusters using the viridis palette
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.7)

# Draw edges
edges = nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight']*5 for u, v in G.edges()], alpha=0.5)

# Draw labels
labels = nx.draw_networkx_labels(G, pos, font_size=10)

plt.title('Network of Shops Based on Rule Similarities (Colored by Hierarchical Clusters)')
plt.axis('off')  # Turn off the axis if not needed
plt.show()



In [ ]:
# Assuming 'feature_matrix_normalized' and 'shops' are already defined
# K-means clustering has been performed with the determined optimal number of clusters
kmeans = KMeans(n_clusters=4, random_state=42)  # Change the number of clusters based on your elbow method results
clusters = kmeans.fit_predict(feature_matrix_normalized)

# Add cluster labels back to the original DataFrame for use in plotting
feature_matrix['Cluster'] = clusters  # Assumes feature_matrix is a DataFrame with shops as rows

# Create a graph
G = nx.Graph()

# Add all nodes for each shop, including cluster info for coloring
for shop, data in feature_matrix.iterrows():
    G.add_node(shop, cluster=data['Cluster'])

# Connect all shops with edges weighted by similarity
for i, shop_i in enumerate(shops):
    for j, shop_j in enumerate(shops):
        if i < j:  # Avoid self-loops and duplicate edges
            similarity = similarity_matrix[i, j]
            G.add_edge(shop_i, shop_j, weight=similarity)

# Calculate node sizes based on the number of unique rules compared to the other shops in their cluster
unique_rules_per_shop = {}
for shop in shops:
    cluster = feature_matrix.at[shop, 'Cluster']
    shops_in_cluster = feature_matrix[feature_matrix['Cluster'] == cluster].index
    all_rules_in_cluster = set()
    for shop_in_cluster in shops_in_cluster:
        all_rules_in_cluster.update(shop_rules[shop_in_cluster])
    unique_rules_per_shop[shop] = len((all_rules_in_cluster) - set(shop_rules[shop])) / len(all_rules_in_cluster)

# Ignore shop 4 for min size calculation
sizes = [unique_rules_per_shop.get(shop, 0) for shop in G.nodes() if shop != 'shop4']
max_size = max(sizes)
min_size = min(sizes)
node_sizes = [(unique_rules_per_shop.get(shop, 0) - min_size) / (max_size - min_size) * 2500 + 100 for shop in G.nodes()]

# Use cluster information for coloring nodes
cluster_colors = [G.nodes[shop]['cluster'] for shop in G.nodes()]

# Draw the network
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=cluster_colors, cmap='viridis', alpha=0.7)
nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight']*5 for u, v in G.edges()], alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10)
plt.title('Network of Shops Based on Rule Similarities (Colored by K-means Clusters)')
plt.axis('off')

# Save the plot as an SVG file
plt.savefig("network_of_shops.svg", format='svg')

# Display the plot
plt.show()

print("Plot saved as network_of_shops.svg")


In [ ]:
# Calculate node sizes based on the number of unique rules compared to the other shops in their cluster
unique_rules_per_shop = {}

for shop in shops:
    cluster = feature_matrix.at[shop, 'Cluster']
    shops_in_cluster = feature_matrix[feature_matrix['Cluster'] == cluster].index
    all_rules_in_cluster = set()
    
    for shop_in_cluster in shops_in_cluster:
        all_rules_in_cluster.update(shop_rules[shop_in_cluster])
    
    unique_rules_per_shop[shop] = len((all_rules_in_cluster) - set(shop_rules[shop])) / len(all_rules_in_cluster)

# Print the unique rules proportion for each shop
for shop, value in unique_rules_per_shop.items():
    print(f"Shop: {shop}\nUnique Rules Proportion: {value:.2f}\n")


In [ ]:
# Calculate node sizes based on the number of unique rules compared to the other shops in their cluster
unique_rules_per_shop = {}
excluded_shop = 'shop7'  # Shop to be excluded

for shop in shops:
    if shop == excluded_shop:
        continue
    
    cluster = feature_matrix.at[shop, 'Cluster']
    shops_in_cluster = feature_matrix[feature_matrix['Cluster'] == cluster].index
    all_rules_in_cluster = set()
    
    for shop_in_cluster in shops_in_cluster:
        if shop_in_cluster != excluded_shop:
            all_rules_in_cluster.update(shop_rules[shop_in_cluster])
    
    unique_rules_per_shop[shop] = len((all_rules_in_cluster) - set(shop_rules[shop])) / len(all_rules_in_cluster)

# Print the unique rules proportion for each shop
for shop, value in unique_rules_per_shop.items():
    print(f"Shop: {shop}\nUnique Rules Proportion: {value:.2f}\n")


In [ ]:
print(len((shop_rules[shop])))

In [ ]:
# Check the keys in shop_rules to ensure the shop IDs are correct
print("Keys in shop_rules:", shop_rules.keys())

# Define shop IDs to analyze
shop_ids = ['shop11', 'shop12', 'shop13']  # Use string keys if needed

# Find the number of rules each shop has
num_rules_per_shop = {shop: len(shop_rules[shop]) for shop in shop_ids}

# Find the rules shared between each pair of shops
shared_rules_11_12 = shop_rules['shop11'].intersection(shop_rules['shop12'])
shared_rules_11_13 = shop_rules['shop11'].intersection(shop_rules['shop13'])
shared_rules_12_13 = shop_rules['shop12'].intersection(shop_rules['shop13'])

# Find the rules shared among all three shops
shared_rules_all = shop_rules['shop11'].intersection(shop_rules['shop12'], shop_rules['shop13'])

# Print the results
print("\nNumber of rules per shop:")
for shop, num_rules in num_rules_per_shop.items():
    print(f"Shop {shop}: {num_rules} rules")

print("\nShared rules between shop pairs:")
print(f"Shared rules between shop 11 and shop 12: {len(shared_rules_11_12)}")
print(f"Shared rules between shop 11 and shop 13: {len(shared_rules_11_13)}")
print(f"Shared rules between shop 12 and shop 13: {len(shared_rules_12_13)}")

print("\nShared rules among all three shops:")
print(f"Shared rules among shop 11, shop 12, and shop 13: {len(shared_rules_all)}")



### Identifying Influential Shops
- **Central Nodes**: Shops located at the center of the network might be influential in terms of customer behavior or product trends. They could be used as models for other shops or as test sites for new products or marketing strategies.
- **Peripheral Nodes**: Shops on the periphery, with fewer connections, might have unique customer demographics or preferences. These could be targeted for more customized approaches.

### Understanding Shop Relationships
- **Cluster Analysis**: Shops colored similarly and clustered together indicate similar customer purchasing patterns. Managers can develop standardized strategies across these shops, such as shared inventory or coordinated marketing campaigns.
- **Cross-Promotion Opportunities**: If two shops are connected but belong to different clusters, there might be an opportunity for cross-promotion or sharing of best practices to align their strategies better.

### Resource Allocation
- **Node Size as an Indicator**: If node size represents the number of unique rules, larger nodes might indicate shops with a broader variety of customer transactions, which could be prioritized for a diverse stock inventory.
- **Investment Prioritization**: Larger and more central shops might be prioritized for investments, renovations, or technology upgrades due to their potential impact on the network.

### Collaborations and Knowledge Sharing
- **Sharing of Best Practices**: Shops with strong connections can be places where knowledge sharing is facilitated, helping to spread best practices across the network.
- **Benchmarking Performance**: Managers can use network structure to set benchmarks or performance goals for shops based on the behavior of their 'neighbors' in the network.

### Strategic Planning
- **Market Positioning**: The network can help in understanding market positioning of each shop, suggesting which shops could be flag bearers for brand image or service quality.
- **Expansion or Restructuring**: For expansion plans or restructuring, shops that occupy central roles or that bridge clusters might be critical to maintain.

### Challenges and Risks
- **Risk Mitigation**: Central shops might be more susceptible to systematic risks (e.g., supply chain disruptions). Understanding their role can help in mitigating these risks.
- **Competition Analysis**: If there’s data on competitors, seeing how your shops cluster in relation to competitors' shops could indicate areas of competitive strength or weakness.

### Personalized Strategies
- **Customized Marketing**: For peripheral shops or those with unique connections, more personalized marketing strategies could be developed to target the specific customer base effectively.

This network graph is a snapshot of the relational dynamics based on rule similarities. It is a strategic tool that can guide various aspects of business planning and operations. Each observation from the graph should be considered in the broader context of business goals, market conditions, and other qualitative data to make informed decisions.



### Reducing Risk
1. **Predictive Analytics for Inventory Management**:
   - **Utilize Confidence**: High confidence rules can predict product demand more accurately. For example, if you find high confidence in the rule {winter coats} → {gloves}, you can stock gloves proportionally to winter coats to meet expected demand without overstocking.
   - **Analyze Seasonal Variations**: Use temporal variations in rule strength to adjust inventory levels seasonally, minimizing the risk of overstocking perishable or seasonal items.

2. **Dynamic Pricing Strategies**:
   - **Employ Lift Analysis**: Products with high lift values can be bundled together at a slight discount to encourage sales while maintaining a higher combined profit margin than selling the items separately at a deeper discount.

### Generating More Revenue
1. **Cross-Selling and Up-Selling**:
   - **Capitalize on Lift and Support**: Identify strong item associations with high lift and support to drive promotional strategies. For instance, place items that are frequently bought together in close proximity, both in-store and on online platforms, to boost impulse purchases.
   - **Tailored Marketing Campaigns**: Use confidence metrics to target customers with personalized marketing, suggesting products they are likely to buy based on their shopping history.

2. **Enhanced Customer Experience**:
   - **Recommendation Systems**: Implement sophisticated recommendation engines that use high-confidence rules to personalize suggestions in real time, enhancing the shopping experience and increasing sales.

### Reduce Costs
1. **Optimized Inventory Management**:
   - **Risk Mitigation via Rule Analysis**: Reduce carrying costs by using predictive analytics from rule confidence to optimize stock levels, ensuring that inventory levels match predicted sales without resulting in dead stock.
   - **Reduce Wastage**: For perishable goods, applying lift and support metrics can help predict optimal stock levels, reducing spoilage and waste.

2. **Efficient Resource Allocation**:
   - **Marketing and Promotions**: Focus marketing efforts and promotional budgets on high-lift, high-support item combinations that are statistically proven to drive sales, ensuring marketing spend yields high returns.
   - **Store Layout Optimization**: Reorganize store layouts based on item association rules to streamline customer flows and improve the shopping experience, potentially reducing labor costs associated with frequently rearranged displays.

### Connecting the Dots: Business Perspective
- **Data-Driven Decisions**: Your analysis empowers businesses to make informed decisions that are backed by data, reducing guesswork and the potential for costly mistakes.
- **Strategic Implementation**: By understanding not only what items are purchased together but also the strength and reliability of these purchases, businesses can strategically target their efforts in areas with the greatest potential return on investment.
- **Balanced Approach**: While generating revenue and reducing costs are direct benefits, reducing risk through predictive insights helps stabilize operations and ensures long-term sustainability.

By incorporating these strategies, businesses can leverage your analytical findings to not only survive in competitive markets but thrive by proactively responding to consumer behavior patterns revealed through your research. This holistic approach to applying association rule mining encapsulates a comprehensive business strategy that optimizes performance across multiple fronts.

To leverage your association rule analysis effectively toward achieving the goals of reducing risk, generating revenue, and preventing loss, here are concise strategies:

### 1. **Reduce Risk**

**Strategy**: Focus on the stability of high confidence rules across different times.
- **Implementation**: Regularly track changes in the confidence levels of your association rules, especially during different seasons or days of the week. Identify rules that maintain high confidence regardless of external factors. Prioritize stability over peak values to mitigate risk associated with volatile market conditions or consumer behaviors.

### 2. **Generate Revenue**

**Strategy**: Capitalize on high lift values to promote products strategically.
- **Implementation**: Use insights from days or seasons with unusually high lift scores to create targeted promotions or bundle offers. For example, if certain product combinations show significantly higher lift during specific seasons, schedule marketing campaigns to promote these bundles during those times to maximize cross-selling opportunities.

### 3. **Prevent Loss**

**Strategy**: Monitor transaction trends to avoid stock-outs and overstock.
- **Implementation**: Analyze transaction count trends across different days of the week and seasons to forecast demand more accurately. Use this data to optimize your inventory levels—increasing stock before predicted high-transaction periods and reducing it when a decrease is anticipated. This approach helps in maintaining a balance, thus preventing both overstock and stock-outs, which are direct contributors to loss.

By systematically applying these strategies, you can create a proactive business model that not only reacts to current trends but also anticipates future changes, thereby securing your business against potential risks, optimizing revenue opportunities, and minimizing losses.

# Low Sales, high Confidence & Lift

In [ ]:
# Function to create a rule string
def create_rule_string(row):
    return f"{row['antecedents']} -> {row['consequents']}"

# Flatten the meta_dfs into a single DataFrame
all_rules_data = []
for shop, years_data in meta_dfs.items():
    for year, df in years_data.items():
        df = df.copy()  # Avoid modifying the original DataFrame
        df['shop'] = shop
        df['year'] = year
        df['rule'] = df.apply(create_rule_string, axis=1)
        df['product_codes'] = df['antecedents'].apply(list) + df['consequents'].apply(list)
        all_rules_data.append(df[['rule', 'shop', 'year', 'lift', 'confidence', 'support', 'product_codes']])

combined_rules_df = pd.concat(all_rules_data, ignore_index=True)

# Explode the DataFrame so each product code is on its own row
exploded_rules_df = combined_rules_df.explode('product_codes')


In [ ]:
# Assuming sales_data is already loaded as shown in your example
# Merge the exploded rules DataFrame with sales_data on product_code, including trx_date and support
merged_df = exploded_rules_df.merge(sales_data[['product_code', 'total_value', 'trx_date']], left_on='product_codes', right_on='product_code', how='left')


In [ ]:
# Aggregate the total sales value for each rule and collect list of transaction dates and support
final_df = merged_df.groupby(['rule', 'shop', 'year', 'lift', 'confidence']).agg(
    total_sales_value=('total_value', 'sum'),
    transaction_dates=('trx_date', list),  # Collect all transaction dates for each rule
    support=('support', 'mean')  # Ensure to capture the support value appropriately
).reset_index()

# Display the DataFrame with rules and their corresponding transaction dates and support
print(final_df[['rule', 'shop', 'year', 'lift', 'confidence', 'support', 'total_sales_value', 'transaction_dates']])


In [ ]:
final_df.head()

In [ ]:
# Set a threshold for 'low sales' as the 25th percentile of the total sales values
low_sales_threshold = final_df['total_sales_value'].quantile(0.1)

# Filter for rules with high confidence (> 0.6) and high lift (> 5) but low sales value
high_performance_rules = final_df[
    (final_df['confidence'] > 0.6) & 
    (final_df['lift'] > 5) & 
    (final_df['total_sales_value'] <= low_sales_threshold)
]

print(high_performance_rules[['rule', 'shop', 'year', 'lift', 'confidence', 'total_sales_value']])


In [ ]:
high_performance_rules.info()

In [ ]:
# Sort the high_performance_rules DataFrame by 'total_sales_value' in ascending order
sorted_high_performance_rules = high_performance_rules.sort_values(by='total_sales_value', ascending=True)

# Display the sorted DataFrame
print(sorted_high_performance_rules[['rule', 'shop', 'year', 'lift', 'confidence', 'total_sales_value']])


In [ ]:
# Set display options to expand the width of the display for a DataFrame column
pd.set_option('display.max_colwidth', None)  # None means no truncation

# Now, when you print the DataFrame, it should show the full content of each 'rule'
print(sorted_high_performance_rules[['rule']])


In [ ]:
def format_rule(rule):
    try:
        antecedents, consequents = rule.split(' -> ')  # ensure the split string matches the actual data
        antecedents = ', '.join(sorted(eval(antecedents)))
        consequents = ', '.join(sorted(eval(consequents)))
        return f"{antecedents} → {consequents}"
    except ValueError as e:
        print(f"Error processing rule: {rule}")
        return None  # or you can return rule to see which rule is causing the problem

# Assuming 'sorted_high_performance_rules' is your DataFrame
# Apply this function to the 'rule' column and store the result in a new column
sorted_high_performance_rules['formatted_rule'] = sorted_high_performance_rules['rule'].apply(format_rule)

# Print each formatted rule
print(sorted_high_performance_rules[['rule', 'formatted_rule']])


In [ ]:
sorted_high_performance_rules.info()

In [ ]:
final_high_performance_rules = sorted_high_performance_rules[['rule', 'shop', 'lift', 'confidence', 'total_sales_value', 'support']]


In [ ]:
final_high_performance_rules.head(25)

# Interpretation

Although we can see some Rules that ahve low Sales and high Sales, this is not transferable to all the shops. I would recommend to go in each shop individually. 

# Meta Analysis, Search 16 Filtered Based on Lift

In [ ]:
def order_id_frozensets(df, columns=['antecedents', 'consequents']):
    """
    Orders the ID frozensets within specified columns of a DataFrame.

    Args:
    df (DataFrame): The DataFrame to process.
    columns (list of str): List of column names to order frozensets in.
    """
    for col in columns:
        # Convert frozenset to sorted list, then back to frozenset
        df[col] = df[col].apply(lambda x: frozenset(sorted(x)))
    return df

# Apply the function to each DataFrame in meta_dfs
for shop, years_dfs_lift in meta_dfs_lift.items():
    for year, df in years_dfs_lift.items():
        meta_dfs_lift[shop][year] = order_id_frozensets(df, columns=['antecedents', 'consequents'])



In [ ]:
# Assuming meta_dfs is your dictionary containing all the shop dataframes for lift
all_data_lift = []  # This will hold all the dataframes
for shop, years in meta_dfs_lift.items():  # Iterating over the meta_dfs dictionary
    for year, df in years.items():  # Iterating over each year dataframe within a shop
        df['shop'] = shop  # Assigning the shop name to a new column
        df['year'] = year  # Assigning the year to a new column
        all_data_lift.append(df)  # Appending the dataframe to the list

# Concatenate all the dataframes into one
combined_data_lift = pd.concat(all_data_lift, ignore_index=True)

# Create 'rule' column by concatenating 'LHS' and 'RHS'
combined_data_lift['rule'] = combined_data_lift['LHS'].astype(str) + ' → ' + combined_data_lift['RHS'].astype(str)


In [ ]:
# Initialize a Counter object to hold frequencies of antecedents and consequents
item_frequencies_lift = Counter()

for shop, years_dfs_lift in meta_dfs_lift.items():
    for year, df in years_dfs_lift.items():
        # Aggregate antecedents
        for items in df['antecedents']:
            item_frequencies_lift.update(items)
        # Aggregate consequents
        for items in df['consequents']:
            item_frequencies_lift.update(items)

# Convert the Counter object to a DataFrame for easier manipulation and visualization

# Create a DataFrame from the Counter object
freq_df_lift = pd.DataFrame(item_frequencies_lift.items(), columns=['Item', 'Frequency']).sort_values(by='Frequency', ascending=False)

# Ensure the 'Item' column is of type string for later operations
freq_df_lift['Item'] = freq_df_lift['Item'].astype(str)


In [ ]:
rule_frequencies_lift = {}

for shop, years_dfs_lift in meta_dfs_lift.items():
    for year, df in years_dfs_lift.items():
        # Iterate through each row to construct sorted rule identifiers
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_lift = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            if rule_identifier_lift not in rule_frequencies_lift:
                rule_frequencies_lift[rule_identifier_lift] = {shop: 1}
            else:
                if shop in rule_frequencies_lift[rule_identifier_lift]:
                    rule_frequencies_lift[rule_identifier_lift][shop] += 1
                else:
                    rule_frequencies_lift[rule_identifier_lift][shop] = 1

# Convert the rule frequencies into a DataFrame for easier manipulation
rules_df_lift = pd.DataFrame.from_dict(rule_frequencies_lift, orient='index').fillna(0)
rules_df_lift.reset_index(inplace=True)
rules_df_lift.rename(columns={'index': 'Rule'}, inplace=True)

# Now, each rule string is sorted within itself by the numeric part of the ID.


In [ ]:
rule_supports_lift = {}

for shop, years_dfs_lift in meta_dfs_lift.items():
    for year, df in years_dfs_lift.items():
        # Iterate through each row to extract rule support
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_lift = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            # Initialize or update the support for the rule in the specific shop
            rule_supports_lift.setdefault(rule_identifier_lift, {})[shop] = row['support']

# Convert the rule supports into a DataFrame for easier manipulation
rules_df_lift = pd.DataFrame.from_dict(rule_supports_lift, orient='index').fillna(0)
rules_df_lift.reset_index(inplace=True)
rules_df_lift.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
 rule_lifts_lift = {}

for shop, years_dfs_lift in meta_dfs_lift.items():
    for year, df in years_dfs_lift.items():
        # Iterate through each row to extract rule lift
        for _, row in df.iterrows():
            # Sort antecedents and consequents
            sorted_antecedents = sorted(list(row['antecedents']), key=lambda x: int(x.split('_')[1]))
            sorted_consequents = sorted(list(row['consequents']), key=lambda x: int(x.split('_')[1]))
            
            # Create a unique identifier for each rule
            rule_identifier_lift = f"{tuple(sorted_antecedents)} -> {tuple(sorted_consequents)}"
            
            # Initialize or update the lift for the rule in the specific shop
            rule_lifts_lift.setdefault(rule_identifier_lift, {})[shop] = row['lift']

# Convert the rule lifts into a DataFrame for easier manipulation
lifts_df_lift = pd.DataFrame.from_dict(rule_lifts_lift, orient='index').fillna(0)
lifts_df_lift.reset_index(inplace=True)
lifts_df_lift.rename(columns={'index': 'Rule'}, inplace=True)


In [ ]:
# Assuming rules_df is already in a suitable format for plotting
plt.figure(figsize=(60, 20))
sns.heatmap(rules_df_lift.set_index('Rule'), annot=True, cmap="YlGnBu", fmt=".2f")
plt.title('Rule Support Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Plotting the heatmap
plt.figure(figsize=(54, 40))
sns.heatmap(rules_df_lift.set_index('Rule'), annot=True, fmt=".0f", cmap="YlGnBu")
plt.title('Rule Frequencies Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Plotting the heatmap
plt.figure(figsize=(62, 22))
sns.heatmap(lifts_df_lift.set_index('Rule'), annot=True, cmap="YlGnBu", fmt=".2f")
plt.title('Rule Lift Across Shops')
plt.ylabel('Rules')
plt.xlabel('Shops')
plt.xticks(rotation=45)
plt.show()

# Meta Analysis, Time Series Search 28

In [ ]:
# Group data by 'rule', 'shop', and 'year'
rule_metrics_time_lift = combined_data_lift.groupby(['rule', 'shop', 'year']).agg({
    'support': 'mean',
    'confidence': 'mean',
    'lift': 'mean'
}).reset_index()

# Calculate the maximum year for each shop and convert it into a DataFrame for merging
max_year_per_shop_lift = rule_metrics_time_lift.groupby('shop')['year'].max().reset_index()
max_year_per_shop_lift.rename(columns={'year': 'year_max'}, inplace=True)

# Merge the maximum year information back into the rule metrics
rule_first_last_year_lift = rule_metrics_time_lift.groupby(['rule', 'shop']).agg(
    first_year=('year', 'min'),
    last_year=('year', 'max')
).reset_index()

rule_first_last_year_lift = rule_first_last_year_lift.merge(max_year_per_shop_lift, on='shop', how='left')
rule_first_last_year_lift['is_current'] = rule_first_last_year_lift['last_year'] == rule_first_last_year_lift['year_max']

# Filter to analyze further or visualize trends
print(rule_first_last_year_lift)


In [ ]:
# Filter out active rules
active_rules_lift = rule_first_last_year_lift[rule_first_last_year_lift['is_current']]

# Printing active rules for verification
print(active_rules_lift)


In [ ]:
# Merge active rules with original metrics to get the full data for these rules
active_rule_metrics_lift = active_rules_lift.merge(rule_metrics_time_lift, on=['rule', 'shop'], how='left')

# Aggregate to get the mean confidence for each rule across all years it was active
active_rule_lift_lift = active_rule_metrics_lift.groupby(['rule', 'shop']).agg({
    'lift': 'mean'
}).reset_index()

# Sorting the results by confidence for better visualization
active_rule_lift_sorted_lift = active_rule_lift_lift.sort_values(by='lift', ascending=False)


In [ ]:
# Setting up the plot
plt.figure(figsize=(10, 20))
sns.barplot(x='lift', y='rule', data=active_rule_lift_sorted_lift, hue='shop', dodge=False)
plt.title('Mean lift of Active Rules by Shop')
plt.xlabel('Mean lift')
plt.ylabel('Rule')
plt.legend(title='Shop')
plt.show()


In [ ]:
# Assuming 'combined_data_lift' is already grouped and aggregated as described earlier

# Prepare the growth_decay DataFrame for lift
growth_decay_lift = rule_first_last_year_lift.copy()

# Calculate growth and decay rates for lift
growth_decay_lift['support_change_lift'] = growth_decay_lift.apply(
    lambda x: (
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['last_year'])]['support'].values[0] -
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['support'].values[0]
    ) / rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['support'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

growth_decay_lift['confidence_change_lift'] = growth_decay_lift.apply(
    lambda x: (
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['last_year'])]['confidence'].values[0] -
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['confidence'].values[0]
    ) / rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['confidence'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

growth_decay_lift['lift_change_lift'] = growth_decay_lift.apply(
    lambda x: (
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['last_year'])]['lift'].values[0] -
        rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['lift'].values[0]
    ) / rule_metrics_time_lift[(rule_metrics_time_lift['rule'] == x['rule']) & (rule_metrics_time_lift['shop'] == x['shop']) & (rule_metrics_time_lift['year'] == x['first_year'])]['lift'].values[0] if x['first_year'] != x['last_year'] else 0,
    axis=1
)

# Now, filter for significant changes
significant_confidence_changes_lift = growth_decay_lift[growth_decay_lift['confidence_change_lift'].abs() > 0.1]
significant_lift_changes_lift = growth_decay_lift[growth_decay_lift['lift_change_lift'].abs() > 0.1]
significant_support_changes_lift = growth_decay_lift[growth_decay_lift['support_change_lift'].abs() > 0.1]



In [ ]:
# Ensure your data is sorted if needed (for better visualization), for example by absolute change
significant_confidence_changes_lift_sorted = significant_confidence_changes_lift.sort_values('confidence_change_lift', ascending=False)

# Visualize significant changes in confidence using the lift-specific data
plt.figure(figsize=(14, 8))  # Adjust the figure size to accommodate the number of rules
sns.barplot(
    x='confidence_change_lift', 
    y='rule', 
    data=significant_confidence_changes_lift_sorted, 
    hue='shop', 
    dodge=False
)
plt.title('Significant Confidence Changes in Rules (Lift Analysis)')
plt.xlabel('Confidence Change Rate')
plt.ylabel('Rule')
plt.grid(True)
plt.legend(title='Shop')
plt.show()


In [ ]:
# Ensure your data is sorted if needed (for better visualization), for example by absolute change
significant_lift_changes_lift_sorted = significant_lift_changes_lift.sort_values('lift_change_lift', ascending=False)

# Visualize significant changes in lift using the lift-specific data
plt.figure(figsize=(10, 8))  # Adjust the figure size if necessary to fit your data
sns.barplot(
    x='lift_change_lift', 
    y='rule', 
    data=significant_lift_changes_lift_sorted, 
    hue='shop', 
    dodge=False
)
plt.title('Significant Lift Changes in Rules')
plt.xlabel('Lift Change Rate')
plt.ylabel('Rule')
plt.grid(True)
plt.legend(title='Shop')
plt.tight_layout()  # Adjust the layout to fit everything nicely
plt.show()


In [ ]:
# Ensure your data is sorted if needed (for better visualization), for example by absolute change
significant_support_changes_lift_sorted = significant_support_changes_lift.sort_values('support_change_lift', ascending=False)

# Visualize significant changes in support using the lift-specific data
plt.figure(figsize=(10, 8))  # Adjust the figure size if necessary to fit your data
sns.barplot(
    x='support_change_lift', 
    y='rule', 
    data=significant_support_changes_lift_sorted, 
    hue='shop', 
    dodge=False
)
plt.title('Significant Support Changes in Rules (Lift Analysis)')
plt.xlabel('Support Change Rate')
plt.ylabel('Rule')
plt.grid(True)
plt.legend(title='Shop')
plt.tight_layout()  # Adjust the layout to fit everything nicely
plt.show()

# Meta Analysis, Network Analysis, Search 29

In [ ]:
# (e.g., frozensets or lists). If they are not, you will need to preprocess these into individual items.

# Create a graph
G = nx.Graph()

# Adding nodes and edges from rules
for idx, row in combined_data_lift.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    for antecedent in antecedents:
        for consequent in consequents:
            if G.has_edge(antecedent, consequent):
                # Increase weight by support or confidence
                G[antecedent][consequent]['weight'] += row['support']
            else:
                G.add_edge(antecedent, consequent, weight=row['support'])

# This creates a graph where nodes are items and edges are weighted by support.


In [ ]:
# Calculate centrality measures
degree_centrality_lift = nx.degree_centrality(G)
betweenness_centrality_lift = nx.betweenness_centrality(G)
closeness_centrality_lift = nx.closeness_centrality(G)

# Convert to DataFrame for easier manipulation and visualization
centrality_df_lift = pd.DataFrame({
    'Node': degree_centrality_lift.keys(),
    'Degree Centrality': degree_centrality_lift.values(),
    'Betweenness Centrality': betweenness_centrality_lift.values(),
    'Closeness Centrality': closeness_centrality_lift.values()
}).sort_values(by='Degree Centrality', ascending=False)

print(centrality_df_lift.head(10))  # Display top 10 central nodes


## Strategic Implications

### Inventory Management: 
Items represented by larger or brighter nodes should be prioritized in inventory decisions as they have the potential to drive more sales when paired with other items.

### Marketing and Promotion: 
These items are ideal candidates for promotions, as their high connectivity suggests they can drive additional purchases. Marketing campaigns can focus on these items to leverage their central role in purchase patterns.

### Product Placement: 
In physical or digital stores, placing these central items in prominent positions can increase visibility and potentially encourage more basket additions.

### Customer Insights: 
Understanding which items are central can help in understanding customer preferences and behaviors, particularly how different products influence shopping patterns.


# Meta Analysis, Aggregated Rules Search 30

In [ ]:
# Count the frequency of each rule across the dataset
rule_frequency_lift = combined_data_lift['rule'].value_counts()

# Filter rules that appear more than a certain threshold, e.g., appear in more than 5 different combinations of shop and year
significant_rules_lift = rule_frequency_lift[rule_frequency_lift > 5]  # Adjust threshold as needed


In [ ]:
# Filter the combined_data to include only significant rules
significant_data_lift = combined_data_lift[combined_data_lift['rule'].isin(significant_rules_lift.index)]

# Calculate average support, confidence, and lift for these significant rules
average_metrics_lift = significant_data_lift.groupby('rule').agg({
    'support': 'mean',
    'confidence': 'mean',
    'lift': 'mean'
}).reset_index()


In [ ]:
# Print the significant rules with their average metrics
print(average_metrics_lift)

# Optionally, save this data to a CSV file
average_metrics_lift.to_csv('significant_rule_metrics.csv', index=False)


In [ ]:
average_metrics_lift.info()

# Meta Analysis, Clustering to see differences, Search 26

In [ ]:
# Example DataFrame structure
data_lift = {
    'shop': combined_data_lift['shop'],
    'rule': combined_data_lift['antecedents'].astype(str) + '->' + combined_data_lift['consequents'].astype(str),
    'lift': combined_data_lift['lift']
}

df_lift = pd.DataFrame(data_lift)

# Pivot table to transform data into a [shops x rules] matrix, filling missing rules with zero
feature_matrix_lift = df_lift.pivot_table(index='shop', columns='rule', values='lift', fill_value=0)

# Normalize the feature matrix
scaler = MinMaxScaler()
feature_matrix_normalized_lift = scaler.fit_transform(feature_matrix_lift)
feature_matrix_normalized_lift = pd.DataFrame(feature_matrix_normalized_lift, index=feature_matrix_lift.index, columns=feature_matrix_lift.columns)


In [ ]:
feature_matrix_lift.head()

In [ ]:
df_lift.head()

In [ ]:
# Determine the optimal number of clusters using the elbow method
sse_lift = {}
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(feature_matrix_normalized_lift)
    sse_lift[k] = kmeans.inertia_  # Sum of squared distances of samples to their closest cluster center

plt.figure()
plt.plot(list(sse_lift.keys()), list(sse_lift.values()))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method For Optimal k")
plt.show()

# Assume the elbow is found at k=4
optimal_k_lift = 4
kmeans = KMeans(n_clusters=optimal_k_lift, random_state=42)
clusters_lift = kmeans.fit_predict(feature_matrix_normalized_lift)

# Add cluster information back to the original DataFrame
feature_matrix_lift['Cluster'] = clusters_lift


In [ ]:
pca_lift = PCA(n_components=2)
principal_components_lift = pca_lift.fit_transform(feature_matrix_normalized_lift)
pca_df_lift = pd.DataFrame(data=principal_components_lift, columns=['PC1_lift', 'PC2_lift'])
pca_df_lift['Cluster'] = clusters_lift

# Assuming 'pca_df_lift' already has PCA results and cluster labels
# Let's add shop identifiers to the DataFrame for easy access
pca_df_lift['Shop'] = feature_matrix_lift.index  # This assumes the index of your feature matrix has shop identifiers

# Now plotting with annotations
plt.figure(figsize=(10, 8))
plot_lift = sns.scatterplot(x='PC1_lift', y='PC2_lift', hue='Cluster', data=pca_df_lift, palette='viridis', s=100, alpha=0.7)

# Adding text annotations for each point
for i in range(pca_df_lift.shape[0]):
    plt.text(x=pca_df_lift.PC1_lift[i] + 0.02,  # x-coordinate position for text
             y=pca_df_lift.PC2_lift[i] + 0.02,  # y-coordinate position for text
             s=pca_df_lift.Shop[i],  # text label
             fontdict=dict(color='black', size=10),
            )

plt.title('PCA Cluster Plot of Shops with Shop Labels (Lift)')
plt.xlabel('Principal Component 1 (Lift)')
plt.ylabel('Principal Component 2 (Lift)')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


1. Data Attributes
The primary attributes used for clustering were based on association rules, specifically:

Rules: Each unique combination of antecedents (items that lead to a purchase) and consequents (items that are purchased as a result) formed a rule.
Confidence: The strength of each rule, which measures the reliability of the rule, was used as the key attribute. Confidence is defined as the probability of seeing the consequent items given the antecedent items have been purchased.

2. Construction of the Feature Matrix
The feature matrix was constructed as follows:

Rows: Each row represented a shop.

Columns: Each column represented a unique association rule derived from the combined data of all shops. The uniqueness of a rule was based on the specific combination of items in the antecedents and consequents.

Values: The value in each cell of the matrix was the confidence level of the rule for that shop. If a shop did not have a particular rule present in its transactions, the confidence was set to zero.

3. Data Preparation for Clustering
Normalization: The feature matrix was normalized to ensure that each rule was equally weighted during the clustering process. Normalization adjusted the confidence values so that they were scaled between 0 and 1 across the dataset. This step is crucial because it prevents rules with naturally higher confidence levels from disproportionately influencing the clustering.

4. Clustering Algorithm
K-Means Clustering: This algorithm was chosen for its efficiency and effectiveness in grouping data into clusters that minimize the variance within each cluster. The number of clusters (k) was determined using the elbow method, which identifies a point where adding more clusters does not significantly improve the within-cluster sum of squares (SSE).

5. Dimensionality Reduction for Visualization
PCA (Principal Component Analysis): Though not directly involved in forming the clusters, PCA was used post-clustering to reduce the high-dimensional feature space into 2 dimensions. This reduction allowed for easy visualization of the clusters and helped interpret how shops are grouped based on the rules' presence and strength.

Conclusion

The clusters were formed based on how similarly shops behaved concerning the association rules prevalent in their transactions. By using confidence as the clustering attribute, the analysis not only considered which rules were common but also how significant these rules were in each shop's context. This approach provided a nuanced view that combined both the qualitative aspect (which rules are present) and the quantitative aspect (how strong these rules are), leading to a comprehensive grouping based on purchasing patterns. This method is particularly useful for understanding market dynamics, tailoring marketing strategies, and optimizing inventory management based on consumer behavior similarities across shops.

In [ ]:
# Suppress specific DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message="is_sparse is deprecated")

# Example DataFrame structure
data_lift = {
    'shop_lift': combined_data_lift['shop'],
    'rule_lift': combined_data_lift['antecedents'].astype(str) + '->' + combined_data_lift['consequents'].astype(str),
    'lift_lift': combined_data_lift['lift']
}

df_lift = pd.DataFrame(data_lift)

# Pivot table to transform data into a [shops x rules] matrix, filling missing rules with zero
feature_matrix_lift = df_lift.pivot_table(index='shop_lift', columns='rule_lift', values='lift_lift', fill_value=0)

# Normalize the feature matrix
scaler_lift = MinMaxScaler()
feature_matrix_normalized_lift = scaler_lift.fit_transform(feature_matrix_lift)
feature_matrix_normalized_lift = pd.DataFrame(feature_matrix_normalized_lift, index=feature_matrix_lift.index, columns=feature_matrix_lift.columns)



In [ ]:
feature_matrix_lift.head()

In [ ]:
df_lift.head()

In [ ]:
# Determine the optimal number of clusters using the elbow method
sse_lift = {}
for k in range(1, 9):
    kmeans_lift = KMeans(n_clusters=k, random_state=42)
    kmeans_lift.fit(feature_matrix_normalized_lift)
    sse_lift[k] = kmeans_lift.inertia_  # Sum of squared distances of samples to their closest cluster center

plt.figure()
plt.plot(list(sse_lift.keys()), list(sse_lift.values()))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method For Optimal k (Lift)")
plt.show()

# Assume the elbow is found at k=3
optimal_k_lift = 3
kmeans_lift = KMeans(n_clusters=optimal_k_lift, random_state=42)
clusters_lift = kmeans_lift.fit_predict(feature_matrix_normalized_lift)

# Add cluster information back to the original DataFrame
feature_matrix_lift['Cluster_lift'] = clusters_lift

In [ ]:
# Assuming 'feature_matrix_normalized_lift' and 'clusters_lift' are already defined

# Perform PCA
pca_lift = PCA(n_components=2)
principal_components_lift = pca_lift.fit_transform(feature_matrix_normalized_lift)
pca_df_lift = pd.DataFrame(data=principal_components_lift, columns=['PC1_lift', 'PC2_lift'])
pca_df_lift['Cluster'] = clusters_lift

# Assuming 'pca_df_lift' already has PCA results and cluster labels
# Let's add shop identifiers to the DataFrame for easy access
pca_df_lift['Shop'] = feature_matrix_lift.index  # This assumes the index of your feature matrix has shop identifiers

# Now plotting with annotations
plt.figure(figsize=(10, 8))
plot_lift = sns.scatterplot(x='PC1_lift', y='PC2_lift', hue='Cluster', data=pca_df_lift, palette='viridis', s=100, alpha=0.7)

# Adding text annotations for each point
for i in range(pca_df_lift.shape[0]):
    plt.text(x=pca_df_lift.PC1_lift[i] + 0.02,  # x-coordinate position for text
             y=pca_df_lift.PC2_lift[i] + 0.02,  # y-coordinate position for text
             s=pca_df_lift.Shop[i],  # text label
             fontdict=dict(color='black', size=10),
            )

plt.title('PCA Cluster Plot of Shops with Shop Labels (Lift)')
plt.xlabel('Principal Component 1 (Lift)')
plt.ylabel('Principal Component 2 (Lift)')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

# Meta Analysis Hierarchical Clustering Search 31

In [ ]:
# Create a custom rule representation using tuples of sorted antecedents and consequents
def create_rule(antecedents, consequents):
    return (tuple(sorted(antecedents)), tuple(sorted(consequents)))

# Create a dictionary to accumulate rules for each shop
shops_lift = combined_data_lift['shop'].unique()
shop_rules_lift = {shop: set() for shop in shops_lift}

for _, row in combined_data_lift.iterrows():
    rule_lift = create_rule(row['antecedents'], row['consequents'])
    shop_rules_lift[row['shop']].add(rule_lift)

# Check rules for Shop11 in shop_rules_lift
print(f"Rules for Shop11 in shop_rules_lift:\n{shop_rules_lift['shop11']}")
print(f"Number of unique rules for Shop11 in shop_rules_lift: {len(shop_rules_lift['shop11'])}")

# Create a binary matrix where each row represents a shop and each column a unique rule
all_rules_lift = list(set.union(*shop_rules_lift.values()))  # Convert the set to a list
rule_matrix_lift = pd.DataFrame(0, index=shops_lift, columns=all_rules_lift, dtype=int)

for shop in shops_lift:
    for rule_lift in shop_rules_lift[shop]:
        rule_matrix_lift.at[shop, rule_lift] = 1

# Convert DataFrame to numpy array for distance calculation
rule_matrix_np_lift = rule_matrix_lift.to_numpy()

# Compute the similarity matrix using Jaccard similarity
similarity_matrix_lift = 1 - pairwise_distances(rule_matrix_np_lift, metric='jaccard')

In [ ]:
# Normalize rule_matrix for hierarchical clustering
scaler = MinMaxScaler()
rule_matrix_scaled_lift = scaler.fit_transform(rule_matrix_lift)

# Hierarchical Clustering
hclust_lift = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hclusters_lift = hclust_lift.fit_predict(rule_matrix_scaled_lift)


In [ ]:
# Generate the linkage matrix
Z = linkage(rule_matrix_scaled_lift, method='ward')

# Plotting the dendrogram
plt.figure(figsize=(15, 10))
dendrogram(Z, labels=shops, leaf_rotation=90, leaf_font_size=12, color_threshold=0)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Shop')
plt.ylabel('Distance')
plt.show()


Here MDS is applied directly to the data and not to the PCA transformed data. 

In [ ]:
# Example: Defining a list of shop identifiers
sorted_shops_lift = ['shop1', 'shop11', 'shop12', 'shop13', 'shop3', 'shop4', 'shop5', 'shop6', 'shop7', 'shop8', 'shop9']
print("Original order of sorted_shops_lift:")
print(sorted_shops_lift)

# Function to extract numbers from shop identifiers
def extract_number(text):
    num = re.search(r'\d+', text)
    return int(num.group()) if num else None

# Sort the list by extracting numbers and sorting based on them
sorted_shops_lift = sorted(sorted_shops_lift, key=extract_number)

print("\nNumerically sorted sorted_shops_lift:")
print(sorted_shops_lift)


In [ ]:
# Example to create a sorted shop list if not already available

# Convert hclusters_lift to a pandas Series with shops as the index
hclusters_series = pd.Series(hclusters_lift, index=sorted_shops_lift)

# Reindex hclusters_series to match pca_df_lift
# This ensures that cluster labels are aligned with the PCA data
hclusters_aligned = hclusters_series.reindex(pca_df_lift['Shop']).values

# Assign the correctly ordered clusters to pca_df_lift
pca_df_lift['Cluster'] = hclusters_aligned

# Now plotting with annotations
plt.figure(figsize=(10, 8))
plot_lift = sns.scatterplot(x='PC1_lift', y='PC2_lift', hue='Cluster', data=pca_df_lift, palette='viridis', s=100, alpha=0.7)

# Adding text annotations for each point
for i in range(pca_df_lift.shape[0]):
    plt.text(x=pca_df_lift.PC1_lift[i] + 0.02,  # x-coordinate position for text
             y=pca_df_lift.PC2_lift[i] + 0.02,  # y-coordinate position for text
             s=pca_df_lift.Shop[i],  # text label
             fontdict=dict(color='black', size=10),
            )

plt.title('PCA Hierarchical Cluster Plot of Shops with Shop Labels (Lift)')
plt.xlabel('Principal Component 1 (Lift)')
plt.ylabel('Principal Component 2 (Lift)')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


In [ ]:
# Create a custom rule representation using tuples of sorted antecedents and consequents
def create_rule(antecedents, consequents):
    return (tuple(sorted(antecedents)), tuple(sorted(consequents)))

# Create a dictionary to accumulate rules for each shop
shops_lift = combined_data_lift['shop'].unique()
shop_rules_lift = {shop: set() for shop in shops_lift}

for _, row in combined_data_lift.iterrows():
    rule_lift = create_rule(row['antecedents'], row['consequents'])
    shop_rules_lift[row['shop']].add(rule_lift)

# Create a binary matrix where each row represents a shop and each column a unique rule
all_rules_lift = list(set.union(*shop_rules_lift.values()))  # Convert the set to a list
rule_matrix_lift = pd.DataFrame(0, index=shops_lift, columns=all_rules_lift, dtype=int)

for shop in shops_lift:
    for rule_lift in shop_rules_lift[shop]:
        rule_matrix_lift.at[shop, rule_lift] = 1

# Convert DataFrame to numpy array for distance calculation
rule_matrix_np_lift = rule_matrix_lift.to_numpy()

# Compute the similarity matrix using Jaccard similarity
similarity_matrix_lift = 1 - pairwise_distances(rule_matrix_np_lift, metric='jaccard')

# Apply MDS
mds_lift = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
shop_positions_lift = mds_lift.fit_transform(similarity_matrix_lift)  # Already 1 - similarity_matrix gives dissimilarity

# Prepare results for plotting
shop_positions_df_lift = pd.DataFrame(shop_positions_lift, index=shops_lift, columns=['x', 'y'])

# Use the cluster information from pca_df_lift
shop_positions_df_lift['MDS Cluster'] = pca_df_lift.set_index('Shop').reindex(shops_lift)['Cluster']

# Create the network graph
G_lift = nx.Graph()

# Add all nodes for each shop
for shop_lift in shops_lift:
    G_lift.add_node(shop_lift)

# Connect all shops with edges weighted by similarity
for i, shop_i_lift in enumerate(shops_lift):
    for j, shop_j_lift in enumerate(shops_lift):
        if i < j:  # Avoid self-loops and duplicate edges
            similarity_lift = similarity_matrix_lift[i, j]
            # Use a continuous scale for edge width
            G_lift.add_edge(shop_i_lift, shop_j_lift, weight=similarity_lift)

# Calculate the number of rules per shop for node sizes
num_rules_per_shop_lift = {shop_lift: len(rules) for shop_lift, rules in shop_rules_lift.items()}

# Normalize the values to use for node size
sizes_lift = [num_rules_per_shop_lift[shop_lift] for shop_lift in G_lift.nodes()]
max_size_lift = max(sizes_lift)
min_size_lift = min(sizes_lift)
node_sizes_lift = [(size_lift - min_size_lift) / (max_size_lift - min_size_lift) * 1000 + 100 for size_lift in sizes_lift]  # Scale between 100 and 1100

# Use the 'MDS Cluster' for coloring nodes
cluster_colors_lift = shop_positions_df_lift['MDS Cluster']

# If the cluster numbers are not sequential starting from 0, map them to a sequential range
unique_clusters_lift = sorted(cluster_colors_lift.unique())
cluster_color_mapping_lift = {cluster_lift: i for i, cluster_lift in enumerate(unique_clusters_lift)}
sequential_cluster_colors_lift = cluster_colors_lift.map(cluster_color_mapping_lift)

# Generate a color palette with seaborn and map the clusters to colors using viridis
palette_lift = sns.color_palette('viridis', len(unique_clusters_lift))
node_colors_lift = [palette_lift[color_lift] for color_lift in sequential_cluster_colors_lift]

# Draw the network with the updated color palette
plt.figure(figsize=(12, 12))
pos_lift = nx.spring_layout(G_lift, seed=42)  # Positions should be consistent with previous plot

# Draw nodes with sizes based on the number of rules and color based on clusters using the viridis palette
nodes_lift = nx.draw_networkx_nodes(G_lift, pos_lift, node_size=node_sizes_lift, node_color=node_colors_lift, alpha=0.7)

# Draw edges
edges_lift = nx.draw_networkx_edges(G_lift, pos_lift, width=[G_lift[u][v]['weight']*5 for u, v in G_lift.edges()], alpha=0.5)

# Draw labels
labels_lift = nx.draw_networkx_labels(G_lift, pos_lift, font_size=10)

plt.title('Network of Shops Based on Rule Similarities (Colored by Hierarchical Clusters) - Lift')
plt.axis('off')  # Turn off the axis if not needed
plt.show()

In [ ]:
# Assuming 'feature_matrix_normalized_lift' and 'shops_lift' are already defined
# K-means clustering has been performed with the determined optimal number of clusters
kmeans_lift = KMeans(n_clusters=3, random_state=42)  # Change the number of clusters based on your elbow method results
clusters_lift = kmeans_lift.fit_predict(feature_matrix_normalized_lift)

# Add cluster labels back to the original DataFrame for use in plotting
feature_matrix_lift['Cluster'] = clusters_lift  # Assumes feature_matrix_lift is a DataFrame with shops as rows

# Create a graph
G_lift = nx.Graph()

# Add all nodes for each shop, including cluster info for coloring
for shop_lift, data in feature_matrix_lift.iterrows():
    G_lift.add_node(shop_lift, cluster=data['Cluster'])

# Connect all shops with edges weighted by similarity
for i, shop_i_lift in enumerate(shops_lift):
    for j, shop_j_lift in enumerate(shops_lift):
        if i < j:  # Avoid self-loops and duplicate edges
            similarity_lift = similarity_matrix_lift[i, j]
            G_lift.add_edge(shop_i_lift, shop_j_lift, weight=similarity_lift)

# Calculate node sizes based on the number of unique rules compared to the other shops in their cluster
unique_rules_per_shop_lift = {}
for shop_lift in shops_lift:
    cluster = feature_matrix_lift.at[shop_lift, 'Cluster']
    shops_in_cluster_lift = feature_matrix_lift[feature_matrix_lift['Cluster'] == cluster].index
    all_rules_in_cluster_lift = set()
    for shop in shops_in_cluster_lift:
        all_rules_in_cluster_lift.update(shop_rules_lift[shop])
    unique_rules_per_shop_lift[shop_lift] = len((all_rules_in_cluster_lift) - set(shop_rules_lift[shop])) / len(all_rules_in_cluster_lift)

sizes_lift = [unique_rules_per_shop_lift[shop_lift] for shop_lift in G_lift.nodes()]
max_size_lift = max(sizes_lift)
min_size_lift = min(sizes_lift)
node_sizes_lift = [(size_lift - min_size_lift) / (max_size_lift - min_size_lift) * 2500 + 100 for size_lift in sizes_lift]

# Use cluster information for coloring nodes
cluster_colors_lift = [G_lift.nodes[shop_lift]['cluster'] for shop_lift in G_lift.nodes()]

# Draw the network
plt.figure(figsize=(12, 12))
pos_lift = nx.spring_layout(G_lift, seed=42)
nx.draw_networkx_nodes(G_lift, pos_lift, node_size=node_sizes_lift, node_color=cluster_colors_lift, cmap='viridis', alpha=0.7)
nx.draw_networkx_edges(G_lift, pos_lift, width=[G_lift[u][v]['weight']*5 for u, v in G_lift.edges()], alpha=0.5)
nx.draw_networkx_labels(G_lift, pos_lift, font_size=10)
plt.title('Network of Shops Based on Rule Similarities (Colored by K-means Clusters)')
plt.axis('off')
plt.show()


In [ ]:
print(len(all_rules_in_cluster))

In [ ]:
# Calculate node sizes based on the number of unique rules compared to the other shops in their cluster
unique_rules_per_shop_lift = {}
excluded_shop = 'shop9'  # Shop to be excluded

for shop in shops:
    if shop == excluded_shop:
        continue
    
    cluster_lift = feature_matrix_lift.at[shop, 'Cluster']
    shops_in_cluster_lift = feature_matrix_lift[feature_matrix_lift['Cluster'] == cluster_lift].index
    all_rules_in_cluster_lift = set()
    
    for shop_in_cluster_lift in shops_in_cluster_lift:
        if shop_in_cluster_lift != excluded_shop:
            all_rules_in_cluster_lift.update(shop_rules_lift[shop_in_cluster_lift])
    
    unique_rules_per_shop_lift[shop] = len((all_rules_in_cluster_lift) - set(shop_rules_lift[shop])) / len(all_rules_in_cluster_lift)

# Print the unique rules proportion for each shop
for shop, value in unique_rules_per_shop_lift.items():
    print(f"Shop: {shop}\nUnique Rules Proportion: {value:.2f}\n")
